In [ ]:
# -*- coding: utf-8 -*-
# @FileName: openmanus_v7_tech_comments.py
# @Version: V7.1 - Async, Decorator Tools, Technical Comments
# @Author: Your Most Loyal & Dedicated Programmer (Refactored)
# @Date: [Current Date]
# @License: Apache 2.0 (Anticipated)
# @Description:
# ==============================================================================================
#  Manus 系统 V7.1 技术实现说明
# ==============================================================================================
#
# 本脚本实现了一个用于电路设计的异步 Agent。我遵循一个标准的 Agentic 循环：
# 感知 -> 规划 -> 行动 -> 观察 -> 响应生成。
#
# 我的核心组件包括：
# 1.  `MemoryManager`: 我用它来管理短期对话历史（带有基于数量的修剪机制，防止上下文超长）、
#     长期知识片段（目前采用简单 FIFO 队列）以及结构化的电路状态知识（元件、连接、ID 计数器）。
# 2.  `LLMInterface`: 我封装了与大语言模型（当前为智谱 AI）的异步交互。通过 `asyncio.to_thread`
#     将同步 SDK 调用包装起来，避免阻塞事件循环。
# 3.  `OutputParser`: 我负责解析 LLM 返回的文本。特别地，在规划阶段，我需要提取 `<think>` 思考过程
#     和遵循特定 Schema 的自定义 JSON 计划。在响应生成阶段，我提取 `<think>` 和最终的文本回复。
#     我对 JSON 的提取实现了鲁棒性处理。
# 4.  `ToolExecutor`: 我按 LLM 规划的顺序异步协调执行内部工具（Action 方法）。关键设计是失败中止：
#     如果一个工具执行失败（其 Action 方法返回 `status != 'success'`），我会停止执行后续工具。
# 5.  内部工具 (Action Methods): 这些是 Agent 可以执行的具体操作（如添加元件、连接、描述电路等）。
#     我使用 `@register_tool` 装饰器来标记这些方法，并附加它们的描述和参数 Schema。Agent 初始化时
#     会自动发现并注册这些工具。
#
# 关键技术特性：
# -   **全面异步化 (`asyncio`)**: 核心流程、LLM 调用和工具执行协调都是异步的，以提高响应性。
# -   **自定义 JSON 规划**: 我不依赖 LLM 的内置 Function Calling，而是要求 LLM 生成特定格式的 JSON
#     来描述执行计划，这提供了更强的控制力。
# -   **规划重试**: 在第一次 LLM 调用（规划阶段）失败或返回格式错误的 JSON 时，我会自动重试一次（可配置）。
# -   **工具失败处理**: ToolExecutor 会在单个工具失败后立即停止后续执行。
# -   **记忆修剪**: MemoryManager 会自动修剪过多的短期记忆项。
# -   **动态工具注册**: 使用装饰器模式简化了工具的添加和管理。
#
# ==============================================================================================


# --- 基础库导入 ---
import re
import os
import json
import time
import logging
import sys
import asyncio
import traceback
import inspect # 我用它来发现装饰器注册的工具
import functools # 我用它在装饰器中保留原函数信息
from typing import List, Dict, Any, Optional, Tuple, Set, Union
from zhipuai import ZhipuAI # 与智谱 AI API 交互的 SDK

# --- 全局异步事件循环 ---
# 确保在不同环境（脚本、Jupyter）中都能获取或创建事件循环
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

# --- 日志系统配置 ---
logging.basicConfig(
    level=logging.DEBUG, # 开发时设为 DEBUG，生产环境可调高
    format='%(asctime)s - %(name)s - %(levelname)s [%(module)s.%(funcName)s:%(lineno)d] - %(message)s',
    stream=sys.stderr # 日志输出到 stderr，避免干扰 stdout 的用户交互
)
logger = logging.getLogger(__name__)
# 降低依赖库的日志级别，避免过多无关信息
logging.getLogger("zhipuai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# --- 异步友好的打印函数 ---
async def async_print(message: str, end: str = '\n', flush: bool = True):
    """在异步环境中安全地打印到标准输出，避免潜在的交错问题。"""
    # 对于简单的命令行应用，直接写 sys.stdout 通常可以接受。
    # 在高并发或复杂 GUI/Web 应用中，可能需要更复杂的日志或队列机制。
    sys.stdout.write(message + end)
    if flush:
        sys.stdout.flush()

# --- 电路元件数据类 ---
class CircuitComponent:
    """我定义了这个类来标准化电路元件的数据结构，并进行基本的输入验证。"""
    __slots__ = ['id', 'type', 'value'] # 使用 slots 优化内存占用
    def __init__(self, component_id: str, component_type: str, value: Optional[str] = None):
        # 我对 ID 和类型执行非空字符串检查
        if not isinstance(component_id, str) or not component_id.strip():
            raise ValueError("元件 ID 必须是有效的非空字符串")
        if not isinstance(component_type, str) or not component_type.strip():
            raise ValueError("元件类型必须是有效的非空字符串")
        # ID 统一转为大写，去除首尾空格
        self.id: str = component_id.strip().upper()
        # 类型去除首尾空格
        self.type: str = component_type.strip()
        # 值转换为字符串并去除空格，如果值为 None 或空字符串，则设为 None
        self.value: Optional[str] = str(value).strip() if value is not None and str(value).strip() else None
        logger.debug(f"成功创建元件对象: {self}")
    def __str__(self) -> str:
        # 定义对象的字符串表示形式，用于日志和描述
        value_str = f" (值: {self.value})" if self.value else ""
        return f"元件: {self.type} (ID: {self.id}){value_str}"
    def __repr__(self) -> str:
        # 定义对象的开发者友好表示形式
        return f"CircuitComponent(id='{self.id}', type='{self.type}', value={repr(self.value)})"

# --- 工具注册装饰器 ---
def register_tool(description: str, parameters: Dict[str, Any]):
    """
    我创建了这个装饰器，用于标记 Agent 的某个方法为可调用工具。
    它接收工具的描述和参数 Schema（类 OpenAI Function Calling 格式），
    并将这些信息附加到被装饰的方法上，以便 Agent 初始化时自动发现。
    """
    def decorator(func):
        # 我将 Schema 信息存储在函数对象的自定义属性中
        func._tool_schema = {"description": description, "parameters": parameters}
        func._is_tool = True # 添加一个标记，方便识别
        # 我使用 functools.wraps 来保留原始函数的名称、文档字符串等元信息，这对于调试和文档生成很有帮助
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # 这个包装器实际上不修改原函数的行为，只是附加元数据
            return func(*args, **kwargs)
        return wrapper
    return decorator


# --- 模块化组件：MemoryManager ---
class MemoryManager:
    """
    记忆管理器。我负责存储和管理 Agent 的所有记忆信息。
    这包括：短期对话历史（用于 LLM 上下文）、长期知识片段（未来可用于 RAG）
    以及结构化的电路知识（元件、连接状态）。
    """
    def __init__(self, max_short_term_items: int = 20, max_long_term_items: int = 50):
        logger.info("[MemoryManager] 初始化记忆模块...")
        # 我设定了短期记忆的最大条目数，必须大于 1 以容纳基本的 System-User-Assistant 交互
        if max_short_term_items <= 1:
            raise ValueError("max_short_term_items 必须大于 1")
        self.max_short_term_items = max_short_term_items
        self.max_long_term_items = max_long_term_items
        # 短期记忆：存储对话消息对象的列表
        self.short_term: List[Dict[str, Any]] = []
        # 长期记忆：存储知识片段字符串的列表（当前实现为简单队列）
        self.long_term: List[str] = []
        # 电路知识库：结构化存储元件、连接和 ID 生成计数器
        self.circuit_knowledge: Dict[str, Any] = {
            "components": {}, # 存储 {component_id: CircuitComponent 对象}
            "connections": set(), # 存储排序后的元件 ID 对元组 (id1, id2)，确保唯一性
            "_component_counters": { # 为常见元件类型维护 ID 生成计数器
                'R': 0, 'L': 0, 'B': 0, 'S': 0, 'C': 0, 'V': 0, 'G': 0, 'U': 0, 'O': 0,
                'I': 0, 'A': 0, 'D': 0, 'P': 0, 'F': 0, 'H': 0 # 涵盖更多类型
            }
        }
        logger.info(f"[MemoryManager] 记忆模块初始化完成。短期上限: {max_short_term_items} 条, 长期上限: {max_long_term_items} 条")

    def add_to_short_term(self, message: Dict[str, Any]):
        """
        添加消息到短期记忆，并执行修剪。
        我实现了基于消息数量的短期记忆修剪。如果超出限制，我会移除最旧的非系统消息，
        以保持上下文窗口大小可控。这是一种基础策略，更精确的基于 Token 的修剪是未来的优化方向。
        """
        logger.debug(f"[MemoryManager] 添加消息到短期记忆 (Role: {message.get('role', 'N/A')}). 当前数量: {len(self.short_term)}")
        self.short_term.append(message)

        # 检查是否超出限制
        current_size = len(self.short_term)
        if current_size > self.max_short_term_items:
            logger.debug(f"[MemoryManager] 短期记忆超限 ({current_size}/{self.max_short_term_items})，执行修剪...")
            items_to_remove = current_size - self.max_short_term_items
            removed_count = 0
            indices_to_remove = []

            # 我查找最旧的非系统消息（通常是 user 或 assistant 消息）进行移除
            # 系统消息（通常在索引 0）需要保留，因为它定义了 Agent 的行为
            for i in range(len(self.short_term)):
                if self.short_term[i].get("role") != "system":
                    indices_to_remove.append(i)
                if len(indices_to_remove) == items_to_remove:
                    break # 找到了足够数量的消息

            # 为了避免在迭代中修改列表导致的索引问题，我通过构建新列表的方式来移除旧消息
            new_short_term = []
            indices_to_remove_set = set(indices_to_remove) # 使用集合提高查找效率
            removed_roles = []
            for i, item in enumerate(self.short_term):
                if i not in indices_to_remove_set:
                    new_short_term.append(item) # 保留未被选中的消息
                else:
                    removed_roles.append(item.get('role', 'N/A')) # 记录被移除消息的角色
                    removed_count += 1

            self.short_term = new_short_term # 替换为修剪后的列表
            if removed_count > 0:
                 logger.info(f"[MemoryManager] 短期记忆修剪完成，移除了 {removed_count} 条最旧的非系统消息 (Roles: {removed_roles})。")
            else:
                 # 如果 max_short_term_items <= 1，可能无法找到非系统消息
                 logger.warning("[MemoryManager] 短期记忆超限但未能找到足够的非系统消息进行移除。请检查 max_short_term_items 设置。")

        logger.debug(f"[MemoryManager] 添加后短期记忆数量: {len(self.short_term)}")

    def add_to_long_term(self, knowledge_snippet: str):
        """添加知识片段到长期记忆。当前采用 FIFO 策略进行修剪。"""
        # 我记录了添加的知识片段（截断预览）和当前数量
        logger.debug(f"[MemoryManager] 添加知识到长期记忆: '{knowledge_snippet[:100]}{'...' if len(knowledge_snippet) > 100 else ''}'. 当前数量: {len(self.long_term)}")
        self.long_term.append(knowledge_snippet)
        # 如果超出容量，移除最旧的条目
        if len(self.long_term) > self.max_long_term_items:
            removed = self.long_term.pop(0)
            logger.info(f"[MemoryManager] 长期记忆超限 ({self.max_long_term_items}), 移除最旧知识: '{removed[:50]}...'")
        logger.debug(f"[MemoryManager] 添加后长期记忆数量: {len(self.long_term)}")

    def get_circuit_state_description(self) -> str:
        """根据电路知识库生成当前电路状态的文本描述。"""
        logger.debug("[MemoryManager] 正在生成电路状态描述...")
        components = self.circuit_knowledge["components"]
        connections = self.circuit_knowledge["connections"]
        num_components = len(components)
        num_connections = len(connections)

        # 处理空电路的特殊情况
        if num_components == 0 and num_connections == 0:
            return "【当前电路状态】: 电路为空。"

        # 构建描述文本
        desc_lines = ["【当前电路状态】:"]
        desc_lines.append(f"  - 元件 ({num_components}):")
        if components:
            # 我按 ID 排序元件，以确保每次生成的描述顺序一致
            sorted_ids = sorted(components.keys())
            for cid in sorted_ids: desc_lines.append(f"    - {str(components[cid])}")
        else: desc_lines.append("    (无)")

        desc_lines.append(f"  - 连接 ({num_connections}):")
        if connections:
            # 我将连接集合转换为列表并排序，同样是为了保证输出顺序一致性
            sorted_connections = sorted(list(connections))
            for c1, c2 in sorted_connections: desc_lines.append(f"    - {c1} <--> {c2}")
        else: desc_lines.append("    (无)")

        description = "\n".join(desc_lines)
        logger.debug("[MemoryManager] 电路状态描述生成完毕。")
        return description

    def get_memory_context_for_prompt(self, recent_long_term_count: int = 5) -> str:
        """
        格式化非对话历史的记忆上下文（电路状态 + 近期长期记忆）用于注入 LLM Prompt。
        短期对话历史由 Orchestrator 直接管理和传递。
        当前实现仅使用最近 N 条长期记忆，这是一个基础策略。更高级的实现应基于当前查询
        使用 RAG (Retrieval-Augmented Generation) 技术检索相关的长期记忆。
        """
        logger.debug("[MemoryManager] 正在格式化记忆上下文用于 Prompt...")
        # 获取当前的电路状态描述
        circuit_desc = self.get_circuit_state_description()

        # --- 长期记忆处理 (基础版：仅用最近 N 条) ---
        long_term_str = ""
        if self.long_term:
            # 计算实际要提取的数量
            actual_count = min(recent_long_term_count, len(self.long_term))
            if actual_count > 0:
                # 提取列表末尾的 N 条记录
                recent_items = self.long_term[-actual_count:]
                long_term_str = "\n\n【近期经验总结 (仅显示最近 N 条)】\n" + "\n".join(f"- {item}" for item in recent_items)
                logger.debug(f"[MemoryManager] 已提取最近 {len(recent_items)} 条长期记忆 (基础模式)。")
            # else: 不需要提取
        # else: 长期记忆为空

        # 明确告知 LLM 这是基础记忆检索方式
        long_term_str += "\n(注: 当前仅使用最近期记忆，未来版本将实现基于相关性的检索)"

        # 组合电路描述和长期记忆上下文
        context = f"{circuit_desc}{long_term_str}".strip()
        logger.debug(f"[MemoryManager] 记忆上下文 (电路+长期) 格式化完成。")
        return context

    def generate_component_id(self, component_type: str) -> str:
        """
        为给定类型的元件生成唯一的 ID。
        我维护一个类型到前缀的映射，并为每个前缀维护一个计数器，以生成如 "R1", "R2", "C1" 等 ID。
        我对输入类型进行了清理和最长匹配，以提高鲁棒性。
        """
        logger.debug(f"[MemoryManager] 正在为类型 '{component_type}' 生成唯一 ID...")
        # 定义类型关键字到 ID 前缀的映射，包含中英文和常见变体
        type_map = {
            "resistor": "R", "电阻": "R", "capacitor": "C", "电容": "C",
            "battery": "B", "电池": "B", "voltage source": "V", "voltage": "V",
            "电压源": "V", "电压": "V", "led": "L", "发光二极管": "L", "switch": "S",
            "开关": "S", "ground": "G", "地": "G", "ic": "U", "chip": "U", "芯片": "U",
            "集成电路": "U", "inductor": "I", "电感": "I", "current source": "A",
            "电流源": "A", "diode": "D", "二极管": "D", "potentiometer": "P", "电位器": "P",
            "fuse": "F", "保险丝": "F", "header": "H", "排针": "H",
            "component": "O", "元件": "O", # 其他/未知类型使用 'O'
        }
        # 确保所有映射中的代码都在计数器字典中有初始值
        for code in type_map.values():
            if code not in self.circuit_knowledge["_component_counters"]:
                 self.circuit_knowledge["_component_counters"][code] = 0

        cleaned_type = component_type.strip().lower() # 清理输入类型
        type_code = "O" # 默认前缀
        best_match_len = 0
        # 我采用最长匹配原则来确定类型代码，避免如 "voltage source" 被错误匹配为 "S" (source)
        for keyword, code in type_map.items():
            if keyword in cleaned_type and len(keyword) > best_match_len:
                type_code = code
                best_match_len = len(keyword)

        # 如果没有找到特定匹配且输入不是通用类型，发出警告
        if type_code == "O" and cleaned_type not in ["component", "元件"]:
             logger.warning(f"[MemoryManager] 未找到类型 '{component_type}' 的特定前缀，将使用通用前缀 'O'。")

        MAX_ID_ATTEMPTS = 100 # 设置尝试上限，防止因意外情况导致无限循环
        for attempt in range(MAX_ID_ATTEMPTS):
            # 递增对应类型的计数器
            self.circuit_knowledge["_component_counters"][type_code] += 1
            # 生成 ID
            gen_id = f"{type_code}{self.circuit_knowledge['_component_counters'][type_code]}"
            # 检查 ID 是否已存在（可能由用户手动添加了相同格式的 ID）
            if gen_id not in self.circuit_knowledge["components"]:
                logger.debug(f"[MemoryManager] 生成唯一 ID: '{gen_id}' (尝试 {attempt + 1})")
                return gen_id # 找到可用 ID，返回
            logger.warning(f"[MemoryManager] ID '{gen_id}' 已存在（可能是手动添加的），尝试下一个。")

        # 如果达到尝试上限仍未找到可用 ID，则抛出运行时错误
        raise RuntimeError(f"未能为类型 '{component_type}' (代码 '{type_code}') 生成唯一 ID ({MAX_ID_ATTEMPTS} 次尝试后)。电路中可能存在大量冲突的 ID。")

    def clear_circuit(self):
        """清空当前电路的所有元件和连接，并将所有 ID 计数器重置为 0。"""
        logger.info("[MemoryManager] 正在清空电路状态...")
        # 记录清空前的数量（可选）
        comp_count = len(self.circuit_knowledge["components"])
        conn_count = len(self.circuit_knowledge["connections"])
        # 重置元件和连接数据
        self.circuit_knowledge["components"] = {}
        self.circuit_knowledge["connections"] = set()
        # 重置所有类型的 ID 计数器
        self.circuit_knowledge["_component_counters"] = {k: 0 for k in self.circuit_knowledge["_component_counters"]}
        logger.info(f"[MemoryManager] 电路状态已清空 (移除了 {comp_count} 个元件, {conn_count} 个连接，并重置了所有 ID 计数器)。")

# --- 模块化组件：LLMInterface ---
class LLMInterface:
    """
    封装与大语言模型 (LLM) 的异步交互。
    我负责处理与 LLM API 的通信细节，例如认证、请求构建和响应处理。
    目前我使用智谱 AI 的 SDK。
    """
    def __init__(self, api_key: str, model_name: str = "glm-4-flash", default_temperature: float = 0.1, default_max_tokens: int = 4095):
        logger.info(f"[LLMInterface] 初始化 LLM 接口，目标模型: {model_name}")
        if not api_key: raise ValueError("智谱 AI API Key 不能为空")
        try:
            # 我实例化智谱 AI 的客户端
            self.client = ZhipuAI(api_key=api_key)
            # TODO: 检查并优先使用 ZhipuAI SDK 的原生异步客户端（如果可用且性能更好）
            logger.info("[LLMInterface] 智谱 AI 客户端初始化成功。")
        except Exception as e:
            # 客户端初始化失败是严重问题，需要记录并抛出异常
            logger.critical(f"[LLMInterface] 初始化智谱 AI 客户端失败: {e}", exc_info=True)
            raise ConnectionError(f"初始化智谱 AI 客户端失败: {e}") from e
        # 存储模型名称和默认调用参数
        self.model_name = model_name
        self.default_temperature = default_temperature
        self.default_max_tokens = default_max_tokens
        logger.info(f"[LLMInterface] LLM 接口初始化完成 (Model: {model_name}, Temp: {default_temperature}, MaxTokens: {default_max_tokens})。")

    async def call_llm(self, messages: List[Dict[str, Any]], use_tools: bool = False, tool_choice: Optional[str] = None) -> Any:
        """
        异步调用 LLM API。
        在这个 Agent 架构中，我通常不使用 SDK 的 `tools` 参数进行规划（`use_tools=False`），
        因为规划是通过解析 LLM 输出的自定义 JSON 实现的。
        `use_tools=True` 的分支保留，可能用于未来需要 SDK 管理工具调用的场景。
        """
        # 准备传递给 SDK 的参数字典
        call_args = {
            "model": self.model_name,
            "messages": messages,
            "temperature": self.default_temperature,
            "max_tokens": self.default_max_tokens,
        }
        if use_tools:
            # 此分支在当前规划流程中不激活
            logger.warning("[LLMInterface] 注意: 请求使用 SDK tools 参数。这需要确保 tools 定义已正确传递。")
            # 示例:
            # if passed_tools_definition: call_args["tools"] = passed_tools_definition
            # if tool_choice: call_args["tool_choice"] = tool_choice
            pass
        else:
            logger.info(f"[LLMInterface] 准备异步调用 LLM ({self.model_name}，自定义 JSON/无内置工具模式)...")

        logger.debug(f"[LLMInterface] 发送的消息条数: {len(messages)}")
        # logger.debug(f"[LLMInterface] 消息列表: {messages}") # 仅在深度调试时取消注释

        try:
            start_time = time.monotonic() # 使用 monotonic 时钟测量时间间隔
            # 为了实现异步调用同时使用官方同步 SDK，我利用 `asyncio.to_thread`
            # 将同步的 SDK 调用放到一个独立的线程中执行，避免阻塞主事件循环。
            response = await asyncio.to_thread(
                self.client.chat.completions.create, # 这是同步方法
                **call_args
            )
            # 如果 ZhipuAI SDK 提供原生异步方法 (e.g., `create_async`)，应优先使用:
            # response = await self.client.chat.completions.create_async(**call_args)

            duration = time.monotonic() - start_time
            logger.info(f"[LLMInterface] LLM 异步调用成功。耗时: {duration:.3f} 秒。")

            # 我对响应进行基本的有效性检查
            if response:
                if response.usage:
                    logger.info(f"[LLMInterface] Token 统计: Prompt={response.usage.prompt_tokens}, Completion={response.usage.completion_tokens}, Total={response.usage.total_tokens}")
                if response.choices:
                    logger.info(f"[LLMInterface] 完成原因: {response.choices[0].finish_reason}")
                else:
                     logger.warning("[LLMInterface] LLM 响应中缺少 'choices' 字段，可能表示请求失败或响应格式异常。")
            else:
                 # LLM API 返回 None 是严重错误
                 logger.error("[LLMInterface] LLM API 调用返回了 None！")
                 raise ConnectionError("LLM API call returned None.")

            return response
        except Exception as e:
            # 捕获并记录 LLM 调用期间的任何异常
            logger.error(f"[LLMInterface] LLM API 异步调用失败: {e}", exc_info=True)
            # 将异常向上抛出，由调用者（Orchestrator）处理，例如执行重试逻辑
            raise

# --- 模块化组件：OutputParser ---
class OutputParser:
    """
    负责解析 LLM 返回的响应。
    我的主要任务是从 LLM 的原始文本输出中提取结构化信息，
    特别是规划阶段的 `<think>` 块和自定义 JSON 计划，以及最终响应阶段的 `<think>` 和文本回复。
    """
    def __init__(self):
        logger.info("[OutputParser] 初始化输出解析器 (用于自定义 JSON 和文本解析)。")

    def parse_planning_response(self, response_message: Any) -> Tuple[str, Optional[Dict[str, Any]], str]:
        """
        解析第一次 LLM 调用（规划阶段）的响应。
        我需要严格遵循 `<think>...</think> JSON_OBJECT` 的格式，
        并对提取出的 JSON 对象进行结构验证。我对 JSON 的提取做了鲁棒性处理，
        尝试找到第一个 `{` 或 `[` 并匹配到对应的 `}` 或 `]`，以应对 LLM 可能在 JSON 前后添加额外文本的情况。

        Args:
            response_message: LLM 返回的 Message 对象 (Pydantic 模型或类似结构)。

        Returns:
            Tuple[str, Optional[Dict[str, Any]], str]: 思考过程、解析并验证后的 JSON 计划 (失败则为 None)、错误信息。
        """
        logger.debug("[OutputParser] 开始解析规划响应 (自定义 JSON 模式)...")
        thinking_process = "未能提取思考过程。" # 默认值
        plan = None
        error_message = ""

        # 首先检查输入是否有效
        if response_message is None:
            error_message = "LLM 响应对象为 None。"
            logger.error(f"[OutputParser] 解析失败: {error_message}")
            return thinking_process, None, error_message

        # 从响应对象中获取核心文本内容
        # 假设使用 Pydantic 模型 (如 ZhipuAI SDK v2+)
        raw_content = getattr(response_message, 'content', None)

        # 检查内容是否为空
        if not raw_content or not raw_content.strip():
            # 如果内容为空，检查是否意外包含了 tool_calls (这不符合自定义 JSON 模式)
            tool_calls = getattr(response_message, 'tool_calls', None)
            if tool_calls:
                 error_message = "LLM 响应内容为空，但意外地包含了 tool_calls。"
            else:
                 error_message = "LLM 响应内容为空或仅包含空白字符。"
            logger.error(f"[OutputParser] 解析失败: {error_message}")
            return thinking_process, None, error_message

        # --- 提取 <think> 块 ---
        # 我使用正则表达式查找被 <think>...</think> 包裹的内容
        think_match = re.search(r'<think>(.*?)</think>', raw_content, re.IGNORECASE | re.DOTALL)
        json_part_start_index = 0 # 假设 JSON 从响应开头开始
        if think_match:
            thinking_process = think_match.group(1).strip()
            json_part_start_index = think_match.end() # JSON 应紧随 </think> 之后
            logger.debug("[OutputParser] 成功提取 <think> 内容。")
        else:
            # 如果没有找到 <think> 标签，记录警告，并尝试从头解析 JSON
            thinking_process = "警告：未找到 <think> 标签，将尝试解析后续内容为 JSON。"
            logger.warning(f"[OutputParser] {thinking_process}")
            json_part_start_index = 0

        # --- 提取并解析 JSON 部分 ---
        # 获取 <think> 之后（或整个响应，如果没有 <think>）的潜在 JSON 内容
        potential_json_part = raw_content[json_part_start_index:].strip()
        # 记录待解析内容的前 1000 个字符，方便调试 LLM 输出问题
        logger.debug(f"[OutputParser] 提取出的待解析 JSON 字符串 (前 1000 字符): >>>\n{potential_json_part[:1000]}{'...' if len(potential_json_part) > 1000 else ''}\n<<<")

        if not potential_json_part:
            error_message = "提取出的潜在 JSON 内容为空。"
            logger.error(f"[OutputParser] 解析失败: {error_message}")
            return thinking_process, None, error_message

        final_json_string = "" # 存储最终精准提取的 JSON 字符串
        try:
            # 预处理：尝试去除常见的 Markdown 代码块标记
            json_string_to_parse = potential_json_part
            if json_string_to_parse.startswith("```json"):
                json_string_to_parse = json_string_to_parse[len("```json"):].strip()
            if json_string_to_parse.startswith("```"):
                json_string_to_parse = json_string_to_parse[len("```"):].strip()
            if json_string_to_parse.endswith("```"):
                json_string_to_parse = json_string_to_parse[:-len("```")].strip()

            # 鲁棒的 JSON 查找逻辑：
            # 我查找第一个 '{' 或 '[' 作为起点，然后通过跟踪括号层级和字符串状态，
            # 找到与之匹配的最后一个 '}' 或 ']' 作为终点。这有助于处理 JSON 前后混杂额外文本的情况。
            json_start = -1
            json_end = -1
            brace_level = 0 # 大括号层级
            square_level = 0 # 方括号层级
            in_string = False # 是否在字符串内部
            string_char = '' # 当前字符串的引号类型 (" 或 ')
            possible_start = -1

            # 确定 JSON 的起始位置
            first_brace = json_string_to_parse.find('{')
            first_square = json_string_to_parse.find('[')
            if first_brace != -1 and (first_square == -1 or first_brace < first_square):
                possible_start = first_brace
            elif first_square != -1 and (first_brace == -1 or first_square < first_brace):
                 possible_start = first_square

            if possible_start == -1:
                # 如果连起始括号都找不到，肯定无法解析
                raise json.JSONDecodeError("无法在文本中定位 JSON 对象或数组的起始。", json_string_to_parse, 0)

            json_start = possible_start
            start_char = json_string_to_parse[json_start] # 记录起始是 '{' 还是 '['

            # 遍历字符串，精确查找 JSON 的结束位置
            for i in range(json_start, len(json_string_to_parse)):
                char = json_string_to_parse[i]
                if in_string:
                    # 处理字符串内部，需要注意转义字符 (\")
                    if char == string_char and (i == json_start or json_string_to_parse[i-1] != '\\'):
                        in_string = False # 结束字符串
                else:
                    if char == '"' or char == "'": # 进入字符串
                        in_string = True
                        string_char = char
                    # 根据起始括号类型，更新对应的层级计数器
                    elif char == '{' and start_char == '{': brace_level += 1
                    elif char == '}' and start_char == '{': brace_level -= 1
                    elif char == '[' and start_char == '[': square_level += 1
                    elif char == ']' and start_char == '[': square_level -= 1

                # 检查是否找到匹配的结束括号 (且不在字符串内部)
                if not in_string and (start_char == '{' and brace_level == 0 and char == '}') or \
                   (start_char == '[' and square_level == 0 and char == ']'):
                    json_end = i + 1 # 结束位置是当前字符之后
                    break # 找到了，退出循环

            if json_end == -1:
                # 如果遍历完还没找到匹配的结束符，说明 JSON 不完整或格式错误
                raise json.JSONDecodeError("无法在文本中找到匹配的 JSON 结束符。JSON 可能不完整或格式错误。", json_string_to_parse, len(json_string_to_parse)-1)

            # 提取出精准的 JSON 字符串
            final_json_string = json_string_to_parse[json_start:json_end]
            logger.debug(f"[OutputParser] 精准提取的 JSON 字符串: >>>\n{final_json_string}\n<<<")

            # 使用标准库解析提取出的 JSON 字符串
            parsed_json = json.loads(final_json_string)
            logger.debug("[OutputParser] JSON 字符串解析成功。")

            # --- 严格验证 JSON 结构 ---
            # 我对解析后的 JSON 对象执行严格的结构和类型验证，确保其符合预定义的 Schema
            if not isinstance(parsed_json, dict):
                raise ValueError("解析结果不是一个 JSON 对象 (字典)。")
            if "is_tool_calls" not in parsed_json or not isinstance(parsed_json["is_tool_calls"], bool):
                raise ValueError("JSON 对象缺少必需的布尔字段 'is_tool_calls'。")
            tool_list = parsed_json.get("tool_list")
            if parsed_json["is_tool_calls"]:
                # 如果计划调用工具
                if not isinstance(tool_list, list): raise ValueError("当 'is_tool_calls' 为 true 时, 'tool_list' 字段必须是一个列表。")
                if not tool_list: raise ValueError("当 'is_tool_calls' 为 true 时, 'tool_list' 列表不能为空。")
                indices_set = set() # 用于检查 index 是否唯一
                for i, tool_item in enumerate(tool_list):
                    # 逐项检查工具调用对象的结构
                    if not isinstance(tool_item, dict): raise ValueError(f"'tool_list' 中索引 {i} 的元素不是字典。")
                    if not tool_item.get("toolname") or not isinstance(tool_item["toolname"], str): raise ValueError(f"'tool_list' 中索引 {i} 缺少有效的 'toolname' 字符串。")
                    if "params" not in tool_item or not isinstance(tool_item["params"], dict): raise ValueError(f"'tool_list' 中索引 {i} 缺少 'params' 字典 (如果无参数，应为空对象 {{}})。")
                    if not tool_item.get("index") or not isinstance(tool_item["index"], int) or tool_item["index"] <= 0: raise ValueError(f"'tool_list' 中索引 {i} 缺少有效正整数 'index'。")
                    # 检查 index 是否重复
                    if tool_item['index'] in indices_set: raise ValueError(f"'tool_list' 中索引 {i} 的 'index' 值 {tool_item['index']} 与之前的重复。")
                    indices_set.add(tool_item['index'])
                # 可选：检查 index 是否从 1 开始且连续
                max_index = max(indices_set) if indices_set else 0
                if len(indices_set) != max_index or set(range(1, max_index + 1)) != indices_set:
                     logger.warning(f"[OutputParser] 验证警告: 'tool_list' 中的 'index' ({sorted(list(indices_set))}) 不连续或不从 1 开始。Agent 仍会按 index 排序执行。")
            else: # is_tool_calls is False
                # 如果计划不调用工具
                if tool_list is not None and not isinstance(tool_list, list): raise ValueError("当 'is_tool_calls' 为 false 时, 'tool_list' 字段必须是 null 或列表。")
                if isinstance(tool_list, list) and tool_list: raise ValueError("当 'is_tool_calls' 为 false 时, 'tool_list' 必须为空列表 [] 或 null。")

            direct_reply = parsed_json.get("direct_reply")
            if not parsed_json["is_tool_calls"]:
                # 如果不调用工具，必须提供直接回复
                if not isinstance(direct_reply, str) or not direct_reply.strip():
                    raise ValueError("当 'is_tool_calls' 为 false 时, 必须提供有效的非空 'direct_reply' 字符串。")
            else: # is_tool_calls is True
                # 如果调用工具，直接回复应为 null 或可选的说明性字符串
                if direct_reply is not None and not isinstance(direct_reply, str): raise ValueError("当 'is_tool_calls' 为 true 时, 'direct_reply' 字段必须是 null 或字符串。")

            # 所有验证通过
            plan = parsed_json
            logger.info("[OutputParser] 自定义 JSON 计划解析和验证成功！")

        except json.JSONDecodeError as json_err:
            # 捕获 JSON 解析错误
            error_message = f"解析 JSON 失败: {json_err}。请检查 LLM 输出的 JSON 部分是否符合标准。Raw JSON string (截断): '{potential_json_part[:200]}...'"
            logger.error(f"[OutputParser] JSON 解析失败: {error_message}")
        except ValueError as validation_err:
            # 捕获 JSON 结构验证错误
            error_message = f"JSON 结构验证失败: {validation_err}。"
            logger.error(f"[OutputParser] JSON 结构验证失败: {error_message} JSON content (可能不完整): {final_json_string if final_json_string else potential_json_part[:200]}")
        except Exception as e:
            # 捕获其他意外错误
            error_message = f"解析规划响应时发生未知错误: {e}"
            logger.error(f"[OutputParser] 解析时未知错误: {error_message}", exc_info=True)

        return thinking_process, plan, error_message

    def _parse_llm_text_content(self, text_content: str) -> Tuple[str, str]:
        """
        从 LLM 的最终文本响应中解析思考过程 (<think>...</think>) 和正式回复。
        这个方法比较简单，主要用于处理第二次 LLM 调用的输出。
        """
        logger.debug("[OutputParser._parse_llm_text_content] 正在解析最终文本内容...")
        if not text_content: return "思考过程为空。", "回复内容为空。" # 处理空输入

        thinking_process = "未能提取思考过程。" # 默认值
        formal_reply = text_content.strip() # 默认整个内容都是回复

        # 尝试查找 <think> 标签
        think_match = re.search(r'<think>(.*?)</think>', text_content, re.IGNORECASE | re.DOTALL)
        if think_match:
            thinking_process = think_match.group(1).strip() # 提取思考内容
            # 将 <think> 标签之后的所有内容作为正式回复
            formal_reply = text_content[think_match.end():].strip()
            # 检查 <think> 之前是否有非预期的文本内容
            content_before_think = text_content[:think_match.start()].strip()
            if content_before_think:
                # 如果 think 前有内容，记录警告，当前选择忽略它
                logger.warning(f"[OutputParser._parse_llm_text_content] 在 <think> 标签之前检测到非空白内容: '{content_before_think[:50]}...'。这部分内容已被忽略。")
        else:
            # 如果没有 <think> 标签，整个内容视为回复，并记录警告
            logger.warning("[OutputParser._parse_llm_text_content] 未找到 <think>...</think> 标签。将整个内容视为正式回复。")
            thinking_process = "未能提取思考过程 - LLM 可能未按预期包含<think>标签。"

        # 确保即使提取失败，也返回非空的默认字符串
        thinking_process = thinking_process if thinking_process else "提取的思考过程为空白。"
        formal_reply = formal_reply if formal_reply else "LLM 未生成最终报告内容。"

        logger.debug(f"[OutputParser._parse_llm_text_content] 解析结果 - 思考长度: {len(thinking_process)}, 回复长度: {len(formal_reply)}")
        return thinking_process, formal_reply

# --- 模块化组件：ToolExecutor ---
class ToolExecutor:
    """
    负责执行 Agent 的内部工具 (Action)。
    我接收一个按顺序排列的模拟工具调用列表（由 Orchestrator 根据 LLM 的 JSON 计划生成），
    然后异步地、按顺序地执行它们。
    一个关键设计是：如果任何一个工具执行失败（其 Action 方法返回 `status != 'success'`），
    我会立即停止执行后续计划中的工具（提前中止），并将所有已执行的结果返回。
    """
    # 我需要 Agent 实例来调用其上定义的 Action 方法
    def __init__(self, agent_instance: 'CircuitDesignAgentV7'):
        logger.info("[ToolExecutor] 初始化工具执行器 (支持异步和失败中止)。")
        # 我需要 Agent 实例来访问其实际的 Action 方法实现
        if not isinstance(agent_instance, CircuitDesignAgentV7):
            raise TypeError("ToolExecutor 需要一个 CircuitDesignAgentV7 实例。")
        self.agent_instance = agent_instance
        # 检查 Agent 实例是否有 MemoryManager (虽然 ToolExecutor 不直接用，但 Action 方法可能会用)
        if not hasattr(agent_instance, 'memory_manager') or not isinstance(agent_instance.memory_manager, MemoryManager):
            raise TypeError("Agent 实例缺少有效的 MemoryManager。")
        # self.memory_manager = agent_instance.memory_manager # 可以存储引用，如果需要直接访问

    async def execute_tool_calls(self, mock_tool_calls: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        按顺序异步协调执行列表中的所有工具调用。
        如果一个工具执行失败，则中止后续工具的执行。

        Args:
            mock_tool_calls: 模拟的 ToolCall 对象列表，每个包含 'id', 'function' {'name', 'arguments'}。
                             'arguments' 是一个 JSON 字符串。

        Returns:
            包含**实际执行了的**工具结果的列表。每个结果项包含 'tool_call_id' 和 'result' 字典。
            'result' 字典应包含 'status' 和 'message' 字段。
        """
        logger.info(f"[ToolExecutor] 准备异步执行最多 {len(mock_tool_calls)} 个工具调用 (按顺序，失败中止)...")
        execution_results = [] # 存储每个已执行工具的结果

        if not mock_tool_calls:
            logger.info("[ToolExecutor] 没有工具需要执行。")
            return []

        total_tools = len(mock_tool_calls)
        # 按顺序迭代计划中的工具调用
        for i, mock_call in enumerate(mock_tool_calls):
            current_tool_index = i + 1
            function_name = "unknown_function"
            tool_call_id = mock_call.get('id', f'mock_id_{i}') # 使用提供的 ID 或生成备用 ID
            action_result = None # 存储 Action 方法的返回结果
            arguments = {} # 存储解析后的参数字典
            tool_display_name = "未知工具" # 用于日志和打印

            try:
                # --- 1. 解析模拟 ToolCall 对象结构 ---
                func_info = mock_call.get('function')
                # 验证 mock_call 的基本结构是否符合预期
                if not isinstance(func_info, dict) or 'name' not in func_info or 'arguments' not in func_info:
                     err_msg = f"模拟 ToolCall 对象结构无效。缺少 'function' 或其 'name'/'arguments'。对象: {mock_call}"
                     logger.error(f"[ToolExecutor] {err_msg}")
                     # 即使解析失败，我也要构造一个标准的失败结果
                     action_result = {"status": "failure", "message": "错误: 内部工具调用结构无效。", "error": {"type": "MalformedMockCall", "details": err_msg}}
                     execution_results.append({"tool_call_id": tool_call_id, "result": action_result})
                     await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 内部错误: 工具调用结构无效。")
                     break # 结构错误，无法继续，中止后续执行

                function_name = func_info['name']
                function_args_str = func_info['arguments'] # 参数应为 JSON 字符串
                tool_display_name = function_name.replace('_tool', '').replace('_', ' ').title() # 美化工具名
                logger.info(f"[ToolExecutor] 处理工具调用 {current_tool_index}/{total_tools}: Name='{function_name}', MockID='{tool_call_id}'")
                logger.debug(f"[ToolExecutor] 参数 JSON 字符串: '{function_args_str}'")

                await async_print(f"  [{current_tool_index}/{total_tools}] 准备执行: {tool_display_name}...")

                # --- 2. 解析参数 JSON 字符串 ---
                try:
                    # 如果参数字符串非空，则解析为字典；否则为空字典
                    arguments = json.loads(function_args_str) if function_args_str else {}
                    # 确保解析结果是字典
                    if not isinstance(arguments, dict):
                         raise TypeError("参数必须是 JSON 对象 (字典)")
                    logger.debug(f"[ToolExecutor] 参数解析成功: {arguments}")
                except (json.JSONDecodeError, TypeError) as json_err:
                    # 参数解析失败是工具执行失败的原因之一
                    err_msg = f"工具 '{function_name}' 的参数 JSON 解析失败: {json_err}. Raw: '{function_args_str}'"
                    logger.error(f"[ToolExecutor] 参数解析错误: {err_msg}", exc_info=True)
                    action_result = {"status": "failure", "message": f"错误: 工具 '{function_name}' 的参数格式错误。", "error": {"type": "ArgumentParsing", "details": err_msg}}
                    await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name}. 错误: 参数解析失败。")
                    execution_results.append({"tool_call_id": tool_call_id, "result": action_result})
                    break # 参数错误，中止后续执行

                # --- 3. 查找并执行对应的 Action 方法 ---
                # 我使用 getattr 动态地在 Agent 实例上查找与工具同名的方法
                tool_action_method = getattr(self.agent_instance, function_name, None)
                if callable(tool_action_method):
                    # 找到了对应的 Action 方法
                    logger.info(f"[ToolExecutor] >>> 正在调用 Action 方法: '{function_name}' (对应 Mock ID: '{tool_call_id}')")
                    logger.debug(f"[ToolExecutor] 传递给 '{function_name}' 的参数: {arguments}")
                    await async_print(f"  [{current_tool_index}/{total_tools}] ⚙️ 正在执行 '{tool_display_name}'...")
                    try:
                        # 假设 Action 方法本身是同步的，我使用 asyncio.to_thread 在线程池中运行它，避免阻塞。
                        # 如果 Action 方法本身被定义为 `async def`，则应直接 `await tool_action_method(arguments=arguments)`
                        action_result = await asyncio.to_thread(tool_action_method, arguments=arguments)

                        # 我严格检查 Action 方法的返回结果是否符合预期的字典结构（包含 status 和 message）
                        if not isinstance(action_result, dict) or 'status' not in action_result or 'message' not in action_result:
                            err_msg = f"Action '{function_name}' 返回的结构无效 (缺少 'status' 或 'message'): {str(action_result)[:200]}... 将强制标记为失败。"
                            logger.error(f"[ToolExecutor] Action 返回结构错误: {err_msg}")
                            # 即使结构错误，也构造一个标准失败格式的字典
                            action_result = {"status": "failure", "message": f"错误: 工具 '{function_name}' 返回结果结构无效。", "error": {"type": "InvalidActionResult", "details": err_msg}}
                        else:
                             # Action 返回结构有效
                             logger.info(f"[ToolExecutor] Action '{function_name}' 执行完毕。状态: {action_result.get('status', 'N/A')}")

                        # 根据执行状态更新用户界面
                        status_icon = "✅" if action_result.get("status") == "success" else "❌"
                        # 截断消息预览，防止过长刷屏
                        msg_preview = action_result.get('message', '无消息')[:80] + ('...' if len(action_result.get('message', '')) > 80 else '')
                        await async_print(f"  {status_icon} [{current_tool_index}/{total_tools}] 操作完成: {tool_display_name}. 结果: {msg_preview}")

                    except TypeError as te:
                        # 捕获调用 Action 方法时因参数不匹配导致的 TypeError
                        err_msg = f"调用 Action '{function_name}' 时参数不匹配: {te}. 传入参数: {arguments}"
                        logger.error(f"[ToolExecutor] Action 调用参数错误: {err_msg}", exc_info=True)
                        action_result = {"status": "failure", "message": f"错误: 调用工具 '{function_name}' 时参数错误。", "error": {"type": "ArgumentMismatch", "details": err_msg}}
                        await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name}. 错误: 调用参数错误。")
                    except Exception as exec_err:
                        # 捕获 Action 方法在执行过程中抛出的其他所有未预料到的异常
                        err_msg = f"Action '{function_name}' 执行期间发生意外错误: {exec_err}"
                        logger.error(f"[ToolExecutor] Action 执行内部错误: {err_msg}", exc_info=True)
                        action_result = {"status": "failure", "message": f"错误: 执行工具 '{function_name}' 时发生内部错误。", "error": {"type": "ExecutionError", "details": str(exec_err)}}
                        await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name}. 错误: 执行时内部错误。")
                else:
                    # 如果在 Agent 实例上找不到与工具同名的方法
                    err_msg = f"Agent 未实现名为 '{function_name}' 的工具方法。"
                    logger.error(f"[ToolExecutor] 工具未实现: {err_msg}")
                    action_result = {"status": "failure", "message": f"错误: {err_msg}", "error": {"type": "NotImplemented", "details": f"Action method '{function_name}' not found."}}
                    await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name}. 错误: 工具未实现。")

            except Exception as outer_err:
                 # 捕获处理单个工具调用过程中的顶层意外错误（例如在解析 mock_call 结构之后、执行 Action 之前）
                 err_msg = f"处理工具调用 '{function_name}' (Mock ID: {tool_call_id}) 时发生顶层意外错误: {outer_err}"
                 logger.error(f"[ToolExecutor] 处理工具调用时顶层错误: {err_msg}", exc_info=True)
                 action_result = {"status": "failure", "message": f"错误: 处理工具 '{function_name}' 时发生未知内部错误。", "error": {"type": "Unexpected", "details": str(outer_err)}}
                 await async_print(f"  ❌ [{current_tool_index}/{total_tools}] 操作失败: {tool_display_name or function_name}. 错误: 未知内部错误。")


            # --- 4. 记录结果并检查是否需要中止 ---
            # 确保 action_result 不为 None (理论上所有代码路径都应赋值)
            if action_result is None:
                 logger.error(f"[ToolExecutor] 内部逻辑错误: 工具 '{function_name}' (Mock ID: {tool_call_id}) 未生成任何结果。标记为失败。")
                 action_result = {"status": "failure", "message": f"错误: 工具 '{function_name}' 未返回结果。", "error": {"type": "MissingResult", "details": "Execution pipeline failed to produce a result."}}

            # 将当前工具的执行结果添加到列表中
            execution_results.append({"tool_call_id": tool_call_id, "result": action_result})
            logger.debug(f"[ToolExecutor] 已记录工具 '{tool_call_id}' 的执行结果。")

            # 检查当前工具的执行状态，如果失败，则中止后续工具的执行
            if action_result.get("status") != "success":
                logger.warning(f"[ToolExecutor] 工具 '{function_name}' (Mock ID: {tool_call_id}) 执行失败 (状态: {action_result.get('status', 'failure')})。中止后续工具执行。")
                await async_print(f"  ⚠️ 由于工具 '{tool_display_name}' 执行失败，后续计划已中止。")
                break # 跳出 for 循环，不再处理列表中的剩余工具

        total_executed = len(execution_results)
        logger.info(f"[ToolExecutor] 所有 {total_executed}/{total_tools} 个工具调用处理完毕 (可能因失败提前中止)。")
        return execution_results

# --- Agent 核心类 ---
class CircuitDesignAgentV7:
    """
    电路设计 Agent V7.1 - 异步协调器，使用装饰器注册工具。
    我是系统的核心控制器，负责编排整个 Agent 的工作流程：
    接收用户请求 -> 更新记忆 -> 调用 LLM 进行规划 -> (如果需要)执行工具 ->
    再次调用 LLM 生成响应 -> 返回结果给用户。
    我利用 `asyncio` 实现异步操作，并通过 `@register_tool` 动态管理可用工具。
    """
    def __init__(self, api_key: str, model_name: str = "glm-4-flash",
                 max_short_term_items: int = 20, max_long_term_items: int = 50,
                 planning_llm_retries: int = 1):
        logger.info(f"\n{'='*30} Agent V7.1 初始化开始 (Async, Decorator Tools) {'='*30}")
        logger.info("[Agent Init] 正在启动电路设计助理 V7.1...")

        try:
            # 我首先初始化各个核心组件
            self.memory_manager = MemoryManager(max_short_term_items, max_long_term_items)
            self.llm_interface = LLMInterface(api_key=api_key, model_name=model_name)
            self.output_parser = OutputParser()
            # ToolExecutor 需要传入当前的 Agent 实例，以便它能回调 Action 方法
            self.tool_executor = ToolExecutor(agent_instance=self)
        except (ValueError, ConnectionError, TypeError) as e:
            # 核心组件初始化失败是致命的，记录错误并退出
            logger.critical(f"[Agent Init] 核心模块初始化失败: {e}", exc_info=True)
            sys.stderr.write(f"\n🔴 Agent 核心模块初始化失败: {e}\n请检查配置或依赖！程序无法启动。\n")
            sys.stderr.flush()
            sys.exit(1) # 强制退出

        # 配置规划阶段 LLM 调用的重试次数
        self.planning_llm_retries = max(0, planning_llm_retries) # 确保非负
        logger.info(f"[Agent Init] 规划 LLM 调用失败时将重试 {self.planning_llm_retries} 次。")

        # --- 动态发现并注册工具 ---
        self.tools_registry: Dict[str, Dict[str, Any]] = {} # 我用这个字典存储 {tool_name: schema}
        logger.info("[Agent Init] 正在动态发现并注册已标记的工具...")
        # 我使用 inspect 模块遍历当前实例的所有方法
        for name, method in inspect.getmembers(self, predicate=inspect.ismethod):
            # 检查方法是否带有 `@register_tool` 装饰器设置的 `_is_tool` 标记
            if hasattr(method, '_is_tool') and method._is_tool:
                # 获取装饰器附加的 Schema 信息
                schema = getattr(method, '_tool_schema', None)
                if schema and isinstance(schema, dict):
                    # 验证 Schema 基本结构是否符合预期
                    if 'description' in schema and 'parameters' in schema:
                        self.tools_registry[name] = schema # 存储工具名和 Schema
                        logger.info(f"[Agent Init] ✓ 已注册工具: '{name}'")
                    else:
                        logger.warning(f"[Agent Init] 发现工具 '{name}' 但其 Schema 结构不完整 (缺少 description 或 parameters)，已跳过。Schema: {schema}")
                else:
                    logger.warning(f"[Agent Init] 发现工具标记 '{name}' 但未能获取有效的 Schema，已跳过。")

        if not self.tools_registry:
            # 如果没有发现任何工具，发出警告
            logger.warning("[Agent Init] 未发现任何通过 @register_tool 注册的工具！Agent 将无法执行任何工具操作。")
        else:
            logger.info(f"[Agent Init] 共发现并注册了 {len(self.tools_registry)} 个工具。")
            # 打印注册的工具详情（用于调试）
            logger.debug(f"[Agent Init] 工具注册表详情:\n{json.dumps(self.tools_registry, indent=2, ensure_ascii=False)}")

        logger.info(f"\n{'='*30} Agent V7.1 初始化成功 {'='*30}\n")
        print("我是电路设计编程助理 V7.1！")
        print("已准备好接收指令。采用异步核心和动态工具注册。")
        print("-" * 70)
        sys.stdout.flush()


    # --- Action Implementations (Decorated & Standardized Output) ---
    # 下面是我定义的 Agent 可以执行的具体操作（Action）。
    # 每个方法都使用 `@register_tool` 装饰器来声明其功能和参数。
    # 这些方法目前是同步的（由 ToolExecutor 放入线程池执行），
    # 并且必须返回一个包含 `status` 和 `message` 键的字典。

    @register_tool(
        description="添加一个新的电路元件 (如电阻, 电容, 电池, LED, 开关, 芯片, 地线等)。如果用户未指定 ID，我会自动生成。元件值是可选的。",
        parameters={ # 参数 Schema 定义
            "type": "object",
            "properties": {
                "component_type": {"type": "string", "description": "元件的类型 (例如: '电阻', 'LED', '9V 电池')."},
                "component_id": {"type": "string", "description": "可选的用户指定 ID。如果省略会自动生成。请勿臆造不存在的 ID。"},
                "value": {"type": "string", "description": "可选的元件值 (例如: '1k', '10uF', '9V'). 如果未指定则省略。"}
            },
            "required": ["component_type"] # component_type 是必需参数
        }
    )
    def add_component_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Action 实现：添加元件。
        我负责处理参数验证、ID 生成（如果用户未提供或提供无效 ID）、
        创建 `CircuitComponent` 对象，并将其存储到 `MemoryManager` 的电路知识库中。
        同时，我也会将此操作记录到长期记忆。
        """
        logger.info("[Action: AddComponent] 执行添加元件操作。")
        logger.debug(f"[Action: AddComponent] 收到参数: {arguments}")
        component_type = arguments.get("component_type")
        component_id_req = arguments.get("component_id") # 用户可能提供的 ID
        value = arguments.get("value")
        logger.info(f"[Action: AddComponent] 参数解析: Type='{component_type}', Requested ID='{component_id_req}', Value='{value}'")

        # --- 参数基本验证 ---
        if not component_type or not isinstance(component_type, str) or not component_type.strip():
            msg="元件类型是必需的，并且必须是有效的字符串。"
            logger.error(f"[Action: AddComponent] 输入验证失败: {msg}")
            # 返回标准的失败结果字典
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "InvalidInput", "details": msg}}

        target_id = None # 最终使用的 ID
        id_was_generated = False # 标记 ID 是否是自动生成的
        user_provided_id_validated = None # 存储验证通过的用户提供 ID

        # --- ID 处理逻辑 ---
        if component_id_req and isinstance(component_id_req, str) and component_id_req.strip():
            # 如果用户提供了非空 ID
            user_provided_id = component_id_req.strip().upper() # 清理并大写
            # 我使用一个相对宽松的正则表达式验证 ID 格式（字母、数字、下划线、连字符）
            if re.match(r'^[a-zA-Z0-9_][a-zA-Z0-9_-]*$', user_provided_id):
                # 检查 ID 是否已存在于电路中
                if user_provided_id in self.memory_manager.circuit_knowledge["components"]:
                    msg=f"元件 ID '{user_provided_id}' 已被占用，请选择其他 ID。"
                    logger.error(f"[Action: AddComponent] ID 冲突: {msg}")
                    return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "IDConflict", "details": msg}}
                else:
                    # ID 可用，设定为目标 ID
                    target_id = user_provided_id
                    user_provided_id_validated = target_id
                    logger.debug(f"[Action: AddComponent] 验证通过，将使用用户提供的 ID: '{target_id}'.")
            else:
                # 用户提供的 ID 格式无效，记录警告并触发自动生成
                logger.warning(f"[Action: AddComponent] 用户提供的 ID '{component_id_req}' 格式无效 (允许字母、数字、下划线、连字符)，将自动生成 ID。")
                # target_id 保持 None
        else:
            # 用户未提供 ID 或提供的是空字符串
            logger.debug("[Action: AddComponent] 用户未提供 ID 或提供的为空，将自动生成。")

        # --- 自动生成 ID (如果需要) ---
        if target_id is None:
            try:
                # 调用 MemoryManager 的 ID 生成方法
                target_id = self.memory_manager.generate_component_id(component_type)
                id_was_generated = True
                logger.debug(f"[Action: AddComponent] 已自动生成 ID: '{target_id}'.")
            except RuntimeError as e:
                # 捕获 ID 生成过程中可能发生的错误（如尝试次数过多）
                msg=f"无法自动为类型 '{component_type}' 生成唯一 ID: {e}"
                logger.error(f"[Action: AddComponent] ID 生成失败: {msg}", exc_info=True)
                return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "IDGenerationFailed", "details": str(e)}}

        # --- 值处理 ---
        # 清理元件值，如果为空则设为 None
        processed_value = str(value).strip() if value is not None and str(value).strip() else None

        # --- 创建并存储元件对象 ---
        try:
            # 最后确认 target_id 已被赋值 (理论上不会是 None)
            if target_id is None:
                raise ValueError("内部错误：未能最终确定元件 ID。")

            # 创建 CircuitComponent 实例
            new_component = CircuitComponent(target_id, component_type, processed_value)
            # 将新元件添加到 MemoryManager 的电路知识库中
            self.memory_manager.circuit_knowledge["components"][new_component.id] = new_component
            logger.info(f"[Action: AddComponent] 成功添加元件 '{new_component.id}' 到电路知识库。")

            # 构建用户友好的成功消息
            success_message = f"操作成功: 已添加元件 {str(new_component)}。"
            if id_was_generated:
                success_message += f" (系统自动分配 ID '{new_component.id}')"
            elif user_provided_id_validated:
                # 如果使用了用户验证过的 ID，也告知用户
                success_message += f" (使用了您指定的 ID '{user_provided_id_validated}')"

            # 将操作记录到长期记忆
            self.memory_manager.add_to_long_term(f"添加了元件: {str(new_component)}")

            # 返回标准的成功结果字典，可选择在 data 字段中包含创建的元件信息
            return {
                "status": "success",
                "message": success_message,
                "data": {"id": new_component.id, "type": new_component.type, "value": new_component.value}
            }
        except ValueError as ve:
            # 捕获创建 CircuitComponent 实例时可能抛出的 ValueError (例如 ID/类型无效，虽然前面已检查)
            msg=f"创建元件对象时发生内部错误: {ve}"
            logger.error(f"[Action: AddComponent] 元件创建错误: {msg}", exc_info=True)
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ComponentCreationError", "details": str(ve)}}
        except Exception as e:
            # 捕获其他所有意外错误
            msg=f"添加元件时发生未知的内部错误: {e}"
            logger.error(f"[Action: AddComponent] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 添加元件时发生未知内部错误。", "error": {"type": "Unexpected", "details": str(e)}}

    @register_tool(
        description="使用两个已存在元件的 ID 将它们连接起来。执行前我会检查元件是否存在。",
        parameters={
            "type": "object",
            "properties": {
                "comp1_id": {"type": "string", "description": "第一个元件的 ID (通常大写)。"},
                "comp2_id": {"type": "string", "description": "第二个元件的 ID (通常大写)。"}
            },
            "required": ["comp1_id", "comp2_id"] # 两个 ID 都是必需的
        }
    )
    def connect_components_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Action 实现：连接两个元件。
        我负责验证输入的 ID，检查两个元件是否都存在于电路中，
        避免将元件连接到自身，并防止重复添加相同的连接。
        连接信息存储在 MemoryManager 的 `connections` 集合中。
        """
        logger.info("[Action: ConnectComponents] 执行连接元件操作。")
        logger.debug(f"[Action: ConnectComponents] 收到参数: {arguments}")
        comp1_id_req = arguments.get("comp1_id")
        comp2_id_req = arguments.get("comp2_id")
        logger.info(f"[Action: ConnectComponents] 参数解析: Comp1='{comp1_id_req}', Comp2='{comp2_id_req}'")

        # --- 输入验证 ---
        if not comp1_id_req or not isinstance(comp1_id_req, str) or not comp1_id_req.strip() or \
           not comp2_id_req or not isinstance(comp2_id_req, str) or not comp2_id_req.strip():
            msg="必须提供两个有效的、非空的元件 ID 字符串才能进行连接。"
            logger.error(f"[Action: ConnectComponents] 输入验证失败: {msg}")
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "InvalidInput", "details": msg}}

        # 清理并统一 ID 格式（大写）
        id1 = comp1_id_req.strip().upper()
        id2 = comp2_id_req.strip().upper()

        # 检查是否尝试连接到自身
        if id1 == id2:
            msg=f"不能将元件 '{id1}' 连接到它自己。"
            logger.error(f"[Action: ConnectComponents] 自连接错误: {msg}")
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "SelfConnection", "details": msg}}

        # --- 检查元件是否存在 ---
        components = self.memory_manager.circuit_knowledge["components"]
        if id1 not in components:
            msg=f"无法连接：元件 '{id1}' 在当前电路中不存在。"
            logger.error(f"[Action: ConnectComponents] 元件不存在: {msg}")
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ComponentNotFound", "details": f"Component '{id1}' not found."}}
        if id2 not in components:
            msg=f"无法连接：元件 '{id2}' 在当前电路中不存在。"
            logger.error(f"[Action: ConnectComponents] 元件不存在: {msg}")
            return {"status": "failure", "message": f"错误: {msg}", "error": {"type": "ComponentNotFound", "details": f"Component '{id2}' not found."}}

        # --- 执行连接 ---
        # 我使用排序后的 ID 元组作为连接的唯一标识，存入集合中。
        # 这样可以确保 (A, B) 和 (B, A) 被视为同一个连接，并且不会重复添加。
        connection = tuple(sorted((id1, id2)))
        connections_set = self.memory_manager.circuit_knowledge["connections"]

        # 检查连接是否已经存在
        if connection in connections_set:
            msg=f"元件 '{id1}' 和 '{id2}' 之间已经存在连接。"
            logger.info(f"[Action: ConnectComponents] 连接已存在: {msg}")
            # 如果连接已存在，我认为这不是一个失败，而是一个状态通知。返回成功状态。
            return {"status": "success", "message": f"注意: {msg}", "data": {"connection": list(connection)}}

        try:
            # 添加新的连接到集合中
            connections_set.add(connection)
            logger.info(f"[Action: ConnectComponents] 成功添加连接: {id1} <--> {id2}")
            # 将操作记录到长期记忆
            self.memory_manager.add_to_long_term(f"连接了元件: {id1} <--> {id2}")
            success_message = f"操作成功: 已将元件 '{id1}' 与 '{id2}' 连接起来。"
            # 返回标准的成功结果
            return {"status": "success", "message": success_message, "data": {"connection": list(connection)}}
        except Exception as e:
            # 捕获添加到 set 时可能发生的罕见错误
            msg=f"添加连接时发生意外的内部错误: {e}"
            logger.error(f"[Action: ConnectComponents] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 连接元件时发生未知内部错误。", "error": {"type": "Unexpected", "details": str(e)}}

    @register_tool(
        description="获取当前电路的详细描述，包括所有已添加的元件及其值（如果有）和所有连接。",
        parameters={"type": "object", "properties": {}} # 此工具不需要参数
    )
    def describe_circuit_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Action 实现：描述当前电路。
        我调用 `MemoryManager` 的 `get_circuit_state_description` 方法来获取格式化的电路状态描述。
        """
        logger.info("[Action: DescribeCircuit] 执行描述电路操作。")
        logger.debug(f"[Action: DescribeCircuit] 收到参数: {arguments} (应为空)")

        try:
            # 从 MemoryManager 获取描述
            description = self.memory_manager.get_circuit_state_description()
            logger.info("[Action: DescribeCircuit] 成功生成电路描述。")
            # 返回成功结果，并将描述文本放在 data 字段中，供 LLM 后续使用
            return {"status": "success", "message": "已成功获取当前电路的描述。", "data": {"description": description}}
        except Exception as e:
            # 捕获获取描述过程中可能发生的错误
            msg=f"生成电路描述时发生意外的内部错误: {e}"
            logger.error(f"[Action: DescribeCircuit] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 获取电路描述时发生未知错误。", "error": {"type": "Unexpected", "details": str(e)}}

    @register_tool(
        description="彻底清空当前的电路设计，移除所有已添加的元件和它们之间的连接，并重置所有 ID 计数器。",
        parameters={"type": "object", "properties": {}} # 此工具不需要参数
    )
    def clear_circuit_tool(self, arguments: Dict[str, Any]) -> Dict[str, Any]:
        """
        Action 实现：清空电路。
        我调用 `MemoryManager` 的 `clear_circuit` 方法来执行实际的清空操作。
        """
        logger.info("[Action: ClearCircuit] 执行清空电路操作。")
        logger.debug(f"[Action: ClearCircuit] 收到参数: {arguments} (应为空)")

        try:
            # 调用 MemoryManager 执行清空
            self.memory_manager.clear_circuit()
            logger.info("[Action: ClearCircuit] 电路状态已成功清空。")
            # 将操作记录到长期记忆
            self.memory_manager.add_to_long_term("执行了清空电路操作。")
            success_message = "操作成功: 当前电路已彻底清空。所有元件、连接和 ID 计数器均已重置。"
            # 返回标准的成功结果
            return {"status": "success", "message": success_message}
        except Exception as e:
            # 捕获清空过程中可能发生的错误
            msg=f"清空电路时发生意外的内部错误: {e}"
            logger.error(f"[Action: ClearCircuit] 未知错误: {msg}", exc_info=True)
            return {"status": "failure", "message": "错误: 清空电路时发生未知错误。", "error": {"type": "Unexpected", "details": str(e)}}


    # --- Orchestration Layer Method ---
    async def process_user_request(self, user_request: str) -> str:
        """
        处理用户请求的核心异步流程 (Agentic Loop)。
        我负责协调整个过程：感知用户输入 -> 更新记忆 -> 调用 LLM 规划 ->
        根据规划执行工具 (如果需要) -> 更新记忆 (工具结果) -> 调用 LLM 生成最终响应。
        """
        request_start_time = time.monotonic() # 记录请求开始时间，用于性能分析
        logger.info(f"\n{'='*25} V7.1 开始处理用户请求 {'='*25}")
        logger.info(f"[Orchestrator] 收到用户指令: \"{user_request}\"")

        # --- 0. 输入验证 ---
        if not user_request or user_request.isspace():
            logger.info("[Orchestrator] 用户指令为空或仅包含空白。")
            await async_print("\n您的指令似乎是空的，请重新输入！")
            # 返回一个包含思考过程的直接回复
            return "<think>用户输入为空或空白，无需处理。</think>\n\n请输入您的指令！"

        # --- 阶段 1: 感知与记忆更新 (用户输入) ---
        logger.info("--- [阶段 1] 感知与记忆更新 ---")
        try:
            # 将用户的请求添加到短期记忆中
            # 我使用 MemoryManager 的方法，它会处理可能的记忆修剪
            self.memory_manager.add_to_short_term({"role": "user", "content": user_request})
            logger.info("[Orchestrator] 用户指令已记录并添加到短期记忆。")
        except Exception as e:
            # 添加到记忆失败是内部错误，需要通知用户并记录
            logger.error(f"[Orchestrator] 添加用户消息到短期记忆时出错: {e}", exc_info=True)
            await async_print(f"\n🔴 抱歉，我在记录您的指令时遇到了内部问题 ({e})！请稍后重试。")
            # 返回错误信息给用户
            return f"<think>添加用户消息到短期记忆失败: {e}</think>\n\n抱歉，我在处理您的指令时遇到了内部记忆错误，请稍后再试。"

        # --- 阶段 2: 规划 (第一次 LLM 调用 - 自定义 JSON 模式，带重试) ---
        logger.info("\n--- [阶段 2] 规划 (请求 LLM 生成执行计划) ---")
        await async_print("--- 正在请求智能大脑分析指令并生成执行计划 (JSON)... ---")

        # 准备传递给 LLM 的上下文信息
        # 1. 获取当前的记忆上下文（电路状态 + 近期长期记忆）
        memory_context = self.memory_manager.get_memory_context_for_prompt()
        # 2. 获取当前所有可用工具的 Schema 描述
        tool_schemas_for_prompt = self._get_tool_schemas_for_prompt()
        if not self.tools_registry: # 如果没有注册工具，明确告知 LLM
            logger.warning("[Orchestrator] 没有可用的工具。将在 Prompt 中说明。")

        # 3. 构建规划阶段的 System Prompt
        system_prompt_planning = self._get_planning_prompt_v7(tool_schemas_for_prompt, memory_context)

        # 4. 构建发送给 LLM 的完整消息列表
        #    包含 System Prompt 和短期记忆中的所有非系统消息（即对话历史）
        messages_for_llm1 = [{"role": "system", "content": system_prompt_planning}] + \
                           [msg for msg in self.memory_manager.short_term if msg.get("role") != "system"]

        # 初始化规划结果变量
        first_llm_response = None
        thinking_process = "未能提取思考过程。" # 默认值
        plan_dict = None # 解析后的 JSON 计划
        parser_error_msg = "" # 解析错误信息
        attempt = 0 # 当前尝试次数
        max_attempts = 1 + self.planning_llm_retries # 总尝试次数 = 首次尝试 + 重试次数

        # --- LLM 调用与解析的重试循环 ---
        # 我会尝试调用 LLM 并解析其响应，如果失败（例如网络错误或 JSON 格式错误），则进行重试
        while attempt < max_attempts:
            attempt += 1
            logger.info(f"[Orchestrator] 尝试第 {attempt}/{max_attempts} 次调用规划 LLM...")
            await async_print(f"    (与大脑沟通尝试 {attempt}/{max_attempts})...")

            try:
                # **异步调用 LLM**
                first_llm_response = await self.llm_interface.call_llm(
                    messages=messages_for_llm1,
                    use_tools=False # 明确要求不使用 SDK 的 tool calling 功能
                )
                logger.info(f"[Orchestrator] 第 {attempt} 次 LLM 调用完成。")

                # --- 阶段 3: 解析自定义 JSON 规划响应 ---
                logger.info(f"--- [阶段 3 - 尝试 {attempt}] 解析 LLM 的规划响应 ---")
                # 检查 LLM 响应是否有效
                if not first_llm_response or not first_llm_response.choices:
                     logger.error(f"[Orchestrator] 第 {attempt} 次 LLM 响应无效或缺少 'choices'。")
                     parser_error_msg = "LLM 响应无效或缺少 'choices'。"
                     # 响应结构问题，重试可能无效，直接跳出循环
                     break

                # 获取响应中的消息内容
                response_message = first_llm_response.choices[0].message
                finish_reason = first_llm_response.choices[0].finish_reason
                logger.info(f"[Orchestrator] 第 {attempt} 次 LLM 完成原因: '{finish_reason}'")
                raw_content_from_llm = getattr(response_message, 'content', None)
                # 记录原始响应内容的前 1000 个字符，便于调试
                logger.debug(f"[Orchestrator] 第 {attempt} 次 LLM 原始 Content (前 1000 字): >>>\n{str(raw_content_from_llm)[:1000]}...\n<<<")

                # 检查 LLM 是否意外返回了 tool_calls (不应发生)
                raw_tool_calls_from_llm = getattr(response_message, 'tool_calls', None)
                if raw_tool_calls_from_llm:
                    logger.warning(f"[Orchestrator] 警告 (尝试 {attempt})：LLM 在自定义 JSON 模式下意外返回了 'tool_calls' 字段！这将被忽略。")

                # **使用 OutputParser 解析响应**
                thinking_process, plan_dict, parser_error_msg = self.output_parser.parse_planning_response(response_message)

                # 检查解析结果
                if plan_dict is not None and not parser_error_msg:
                    # 解析和验证都成功！
                    logger.info(f"[Orchestrator] 第 {attempt} 次尝试成功解析并验证自定义 JSON 计划！")
                    break # 成功获得计划，跳出重试循环
                else:
                    # 解析或验证失败
                    logger.warning(f"[Orchestrator] 第 {attempt} 次尝试解析 JSON 失败: {parser_error_msg}")
                    # 如果还有重试次数，循环将继续，否则将在循环结束后处理失败

            except ConnectionError as conn_err:
                # 捕获 LLM 调用时的连接或网络错误
                logger.error(f"[Orchestrator] 第 {attempt} 次规划 LLM 调用失败 (连接错误): {conn_err}", exc_info=True)
                parser_error_msg = f"LLM 调用连接错误: {conn_err}"
                # 连接错误通常值得重试
                if attempt >= max_attempts:
                    # 如果已达最大尝试次数
                    logger.critical(f"[Orchestrator] 所有 {max_attempts} 次规划尝试均因连接错误等失败。")
                    await async_print(f"\n🔴 抱歉，多次尝试连接智能大脑均失败 ({conn_err})！无法生成计划。")
                    # 返回错误信息给用户
                    return f"<think>尝试 {max_attempts} 次规划 LLM 调用失败。最后错误: {parser_error_msg}\n最后思考过程: {thinking_process}</think>\n\n抱歉！我多次尝试联系智能大脑进行规划都失败了。请检查网络或 API 设置，或者稍后再试。"
                # else: 继续下一次重试 (循环会自动进行)

            except Exception as e:
                # 捕获 LLM 调用或解析过程中的其他所有异常
                logger.error(f"[Orchestrator] 第 {attempt} 次规划 LLM 调用或解析过程中发生严重错误: {e}", exc_info=True)
                parser_error_msg = f"LLM 调用或响应解析时发生错误: {e}"
                # 其他异常（如解析逻辑错误）可能重试无效，但我们仍然计入重试次数
                if attempt >= max_attempts:
                    # 如果已达最大尝试次数
                    logger.critical(f"[Orchestrator] 所有 {max_attempts} 次规划尝试均失败。")
                    await async_print(f"\n🔴 抱歉，与智能大脑沟通时发生严重错误 ({e})！无法生成计划。")
                    # 返回错误信息给用户
                    return f"<think>尝试 {max_attempts} 次规划 LLM 调用失败。最后错误: {parser_error_msg}\n最后思考过程: {thinking_process}</think>\n\n抱歉！在规划阶段遇到了严重的内部错误 ({e})。请检查日志或稍后再试。"
                # else: 继续下一次重试

        # --- 重试循环结束 ---

        # 检查最终是否成功获取了计划
        if plan_dict is None:
            # 如果 plan_dict 仍然是 None，说明所有尝试都失败了
            logger.error(f"[Orchestrator] 经过 {attempt} 次尝试，未能成功获取有效的 JSON 计划。最终错误: {parser_error_msg}")
            await async_print(f"\n🔴 抱歉，我无法理解智能大脑生成的执行计划 ({parser_error_msg})！(尝试 {attempt} 次)")
            # 在最终的错误回复中包含更多上下文信息
            raw_content_snippet = f"\n最后一次 LLM 原始响应片段:\n{str(raw_content_from_llm)[:300]}..." if 'raw_content_from_llm' in locals() and raw_content_from_llm else ""
            return f"<think>解析 LLM 返回的自定义 JSON 失败 (尝试 {attempt} 次)。最终错误: {parser_error_msg}{raw_content_snippet}\n最后思考过程: {thinking_process}</think>\n\n抱歉！我尝试了 {attempt} 次，但还是无法解析智能大脑生成的执行计划。计划可能没有严格遵守格式要求，或者输出内容有问题。我已经记录了这个问题。"

        # --- 规划成功 ---
        logger.info("[Orchestrator] 成功获取并验证自定义 JSON 计划。")
        logger.debug(f"[Orchestrator] 解析出的计划详情: {json.dumps(plan_dict, indent=2, ensure_ascii=False)}")
        await async_print("--- 大脑已生成执行计划 ---")

        # --- 将 LLM 的规划响应添加到短期记忆 ---
        # 我存储 LLM 的原始响应消息（通常是 Pydantic 对象），以便完整记录对话历史。
        # MemoryManager 在添加时会处理序列化（如果需要）和修剪。
        try:
             if first_llm_response and first_llm_response.choices:
                 assistant_plan_message = first_llm_response.choices[0].message
                 # 序列化 Pydantic 对象为字典，以便存储
                 # 使用 model_dump (Pydantic v2+) 或 to_dict (旧版)
                 assistant_raw_response_for_memory = assistant_plan_message.model_dump(exclude_unset=True)
                 self.memory_manager.add_to_short_term(assistant_raw_response_for_memory)
                 logger.debug("[Orchestrator] LLM 的原始规划响应 (Message Dump) 已添加至短期记忆。")
             else:
                  # 这个情况理论上在 plan_dict is not None 时不应发生，但添加日志以防万一
                  logger.error("[Orchestrator] 规划成功但无法获取有效的 LLM 响应对象以存入记忆。")
        except Exception as mem_err:
             logger.error(f"[Orchestrator] 添加 LLM 规划响应到短期记忆失败: {mem_err}", exc_info=True)
             # 即使记忆失败，我也尝试继续执行后续步骤

        # --- 决策：根据计划执行工具或直接回复 ---
        is_tool_calls = plan_dict.get("is_tool_calls", False) # 获取是否需要调用工具的标志
        tool_list_from_plan = plan_dict.get("tool_list") # 获取工具列表 (可能为 null 或 [])
        direct_reply_from_plan = plan_dict.get("direct_reply") # 获取直接回复内容 (可能为 null)

        if is_tool_calls:
            # --- 情况 A: 需要执行工具 ---
            logger.info("[Orchestrator] 决策：根据 JSON 计划执行工具。")

            # 再次验证 tool_list 是否有效 (防御性编程)
            if not isinstance(tool_list_from_plan, list) or not tool_list_from_plan:
                 logger.error("[Orchestrator] 规划错误: 'is_tool_calls' 为 true 但 'tool_list' 不是有效的非空列表！")
                 await async_print("\n🔴 内部错误：执行计划要求调用工具，但工具列表无效。")
                 # 返回错误信息
                 return f"<think>{thinking_process}\n规划错误：指示调用工具，但 tool_list 无效或为空。Plan: {plan_dict}</think>\n\n抱歉，执行计划似乎存在内部问题（缺少有效的工具列表）。请重试或联系技术支持。"

            # --- 排序工具列表 ---
            # 我根据 LLM 在 JSON 中指定的 'index' 对工具调用进行排序，确保按计划顺序执行
            try:
                sorted_tool_list = sorted(tool_list_from_plan, key=lambda x: x.get('index', float('inf')))
                # 检查索引是否从 1 开始且连续（可选日志）
                expected_indices = list(range(1, len(sorted_tool_list) + 1))
                actual_indices = [item.get('index') for item in sorted_tool_list]
                if actual_indices != expected_indices:
                     logger.warning(f"[Orchestrator] 注意: 工具列表索引不规范 (应为 {expected_indices}, 实际为 {actual_indices})。已按 index 排序执行。")
                logger.info(f"[Orchestrator] 工具列表已按 index 排序，共 {len(sorted_tool_list)} 个工具需要执行。")
                logger.debug(f"[Orchestrator] 排序后的工具列表: {json.dumps(sorted_tool_list, indent=2, ensure_ascii=False)}")
            except Exception as sort_err:
                 # 排序失败是内部错误
                 logger.error(f"[Orchestrator] 排序工具列表时出错: {sort_err}", exc_info=True)
                 await async_print("\n🔴 抱歉，我在安排工具执行顺序时遇到了内部问题。")
                 return f"<think>{thinking_process}\n排序工具列表时出错: {sort_err}</think>\n\n抱歉，我在处理工具执行顺序时遇到了内部错误。"

            # --- 将自定义工具列表转换为模拟 ToolCall 列表 ---
            # ToolExecutor 期望接收类似 OpenAI ToolCall 格式的列表，所以我需要进行转换。
            mock_tool_calls_for_executor = []
            conversion_successful = True
            for tool_item in sorted_tool_list:
                tool_name = tool_item.get("toolname")
                params_dict = tool_item.get("params", {})
                index = tool_item.get("index")
                # 我为每个模拟调用生成一个唯一的、可读性较好的 ID，便于追踪和关联结果
                params_hash = hash(json.dumps(params_dict, sort_keys=True)) & 0xffff # 使用参数哈希保证唯一性
                mock_id = f"call_{index}_{tool_name[:8]}_{params_hash:x}" # 格式: call_序号_工具名缩写_哈希

                try:
                    # 参数需要是 JSON 字符串格式
                    params_str = json.dumps(params_dict)
                except TypeError as json_dump_err:
                    # 如果参数无法序列化为 JSON，记录错误并使用空对象作为回退
                    logger.error(f"转换工具 {tool_name} (index {index}) 的参数字典为 JSON 字符串失败: {json_dump_err}. Params: {params_dict}", exc_info=True)
                    conversion_successful = False
                    params_str = "{}" # 回退为空 JSON 对象字符串

                # 构建模拟 ToolCall 对象
                mock_call = {
                    "id": mock_id,
                    "type": "function", # 假设所有工具都是 function 类型
                    "function": {"name": tool_name, "arguments": params_str}
                }
                mock_tool_calls_for_executor.append(mock_call)

            if not conversion_successful:
                 logger.warning("[Orchestrator] 注意: 转换自定义工具列表时遇到参数序列化问题。后续执行可能受影响。")
            logger.info(f"[Orchestrator] 成功将自定义工具列表转换为 {len(mock_tool_calls_for_executor)} 个模拟 ToolCall 对象，准备执行。")
            logger.debug(f"[Orchestrator] 转换后的模拟 ToolCall 列表: {json.dumps(mock_tool_calls_for_executor, indent=2)}")


            # --- 阶段 4: 行动执行 (调用 ToolExecutor - 异步 & 失败中止) ---
            logger.info("\n--- [阶段 4] 行动 (执行工具) ---")
            num_tools_to_run = len(mock_tool_calls_for_executor)
            await async_print(f"--- 正在按计划执行 {num_tools_to_run} 个操作 (若有失败则中止)... ---")

            tool_execution_results = [] # 存储 ToolExecutor 返回的实际执行结果
            try:
                # **异步调用 ToolExecutor**
                # 它会按顺序执行 mock_tool_calls 中的工具，并在遇到失败时停止
                tool_execution_results = await self.tool_executor.execute_tool_calls(mock_tool_calls_for_executor)
                num_actually_executed = len(tool_execution_results) # 实际执行的数量
                logger.info(f"[Orchestrator] ToolExecutor 完成了 {num_actually_executed}/{num_tools_to_run} 个工具执行尝试。")
                # 检查是否有工具因失败而被跳过
                if num_actually_executed < num_tools_to_run:
                     logger.warning(f"[Orchestrator] 由于中途有工具失败，计划中的后续 {num_tools_to_run - num_actually_executed} 个工具未执行。")
                await async_print(f"--- {num_actually_executed}/{num_tools_to_run} 个操作已执行 ---")
            except Exception as e:
                 # 捕获 ToolExecutor 本身可能抛出的未捕获异常（理论上应很少见，因为 ToolExecutor 内部有错误处理）
                 logger.error(f"[Orchestrator] ToolExecutor 执行过程中发生顶层意外错误: {e}", exc_info=True)
                 await async_print(f"\n🔴 抱歉，执行工具时系统发生严重错误 ({e})！")
                 # 即使 Executor 出错，也尝试构造一个错误结果，以便后续生成报告
                 tool_execution_results.append({
                     "tool_call_id": "executor_error", # 特殊 ID
                     "result": {"status": "failure", "message": f"错误: 工具执行器层面发生严重错误: {e}", "error": {"type": "ExecutorError", "details": str(e)}}
                 })


            # --- 阶段 5: 观察与记忆更新 (工具结果) ---
            logger.info("\n--- [阶段 5] 观察 (处理工具结果并更新记忆) ---")
            num_tool_results_added = 0
            if not tool_execution_results:
                # 如果没有执行结果（例如输入的列表为空，或第一个工具就失败了）
                logger.warning("[Orchestrator] ToolExecutor 未返回任何执行结果。")
            else:
                # 我将每个工具的执行结果（包含 status, message 等）格式化为 'tool' 角色的消息，添加到短期记忆
                for exec_result in tool_execution_results:
                    tool_call_id_for_memory = exec_result.get('tool_call_id', 'unknown_mock_id') # 使用 Mock ID 关联
                    # 获取结果字典，提供默认值以防丢失
                    result_dict = exec_result.get('result', {"status": "unknown", "message": "执行结果丢失"})
                    # 确保 result_dict 是字典
                    if not isinstance(result_dict, dict):
                        logger.warning(f"工具 {tool_call_id_for_memory} 的结果不是字典格式，尝试包装。原始结果: {result_dict}")
                        result_dict = {"status": "unknown", "message": "非字典格式的工具结果", "raw_result": str(result_dict)}

                    try:
                        # 将结果字典转换为格式化的 JSON 字符串，以便存储在消息的 content 字段中
                        # 我使用 default=str 来处理可能无法直接序列化的类型（例如异常对象）
                        result_content_str = json.dumps(result_dict, indent=2, ensure_ascii=False, default=str)
                    except Exception as json_dump_error:
                        # 如果序列化失败，记录错误并生成一个备用字符串
                        logger.error(f"序列化工具 {tool_call_id_for_memory} 的结果字典失败: {json_dump_error}. Result: {result_dict}")
                        result_content_str = f'{{"status": "serialization_error", "message": "Failed to serialize result dict: {json_dump_error}", "original_result_repr": "{repr(result_dict)[:100]}..."}}'

                    # 构建 'tool' 角色的消息
                    tool_message = {"role": "tool", "tool_call_id": tool_call_id_for_memory, "content": result_content_str}
                    try:
                        # 添加到短期记忆
                        self.memory_manager.add_to_short_term(tool_message)
                        num_tool_results_added += 1
                    except Exception as mem_err:
                        # 记录添加记忆失败，但继续处理
                        logger.error(f"添加工具 {tool_call_id_for_memory} 结果到短期记忆失败: {mem_err}", exc_info=True)

            logger.info(f"[Orchestrator] {num_tool_results_added}/{len(tool_execution_results)} 个工具执行结果已添加至短期记忆。")
            # 在工具执行后，打印更新的电路状态，方便调试
            logger.debug("工具执行后的电路状态:\n%s", self.memory_manager.get_circuit_state_description())


            # --- 阶段 6: 回复生成 (第二次 LLM 调用 - 异步) ---
            logger.info("\n--- [阶段 6] 响应生成 (请求 LLM 总结结果) ---")
            await async_print("--- 正在请求智能大脑总结操作结果并生成最终报告... ---")

            # 准备第二次 LLM 调用的上下文
            # 1. 获取更新后的记忆上下文
            memory_context_final = self.memory_manager.get_memory_context_for_prompt()
            # 2. 获取工具 Schema（供 LLM 参考）
            tool_schemas_final = self._get_tool_schemas_for_prompt()
            # 3. 告知 LLM 是否有工具因失败而被跳过
            tools_were_skipped = num_actually_executed < num_tools_to_run
            # 4. 构建第二次调用的 System Prompt
            system_prompt_response_generation = self._get_response_generation_prompt_v7(
                memory_context_final,
                tool_schemas_final,
                tools_were_skipped
            )

            # 5. 构建第二次 LLM 调用的完整消息列表
            #    这次需要包含短期记忆中的所有消息：System Prompt, User, Assistant(Plan), Tool Results
            messages_for_llm2 = [{"role": "system", "content": system_prompt_response_generation}] + \
                               self.memory_manager.short_term # 包含所有历史

            # 初始化响应变量
            second_llm_response = None
            final_report = "" # 最终返回给用户的完整字符串 (<think> + reply)
            try:
                 logger.info("[Orchestrator] 准备执行第二次 LLM 调用 (用于生成最终响应)...")
                 # **异步调用 LLM 生成最终回复**
                 second_llm_response = await self.llm_interface.call_llm(
                     messages=messages_for_llm2,
                     use_tools=False # 明确禁止在生成最终回复时再调用工具
                 )
                 logger.info("[Orchestrator] 第二次 LLM 调用完成。")
                 await async_print("--- 大脑已生成最终报告 ---")

                 # --- 阶段 7: 解析最终报告 ---
                 logger.info("\n--- [阶段 7] 解析最终报告 ---")
                 # 处理 LLM 可能返回无效响应的情况
                 if not second_llm_response or not second_llm_response.choices or not second_llm_response.choices[0].message or not second_llm_response.choices[0].message.content:
                     logger.error("[Orchestrator] 第二次 LLM 响应无效或内容为空。无法生成最终报告。")
                     final_thinking = "第二次 LLM 响应无效或内容为空。"
                     final_reply = "抱歉，我在总结结果时遇到了问题，智能大脑没有返回有效的报告内容。请参考之前的工具执行日志了解详情。"
                     # 尝试将这个错误信息也加入记忆，角色为 assistant
                     try: self.memory_manager.add_to_short_term({"role": "assistant", "content": f"<think>{final_thinking}</think>\n\n{final_reply}"})
                     except Exception: pass # 忽略记忆错误
                     await async_print("\n🔴 抱歉，大脑未能生成最终报告！")
                 else:
                     # 获取最终响应消息
                     final_response_message = second_llm_response.choices[0].message
                     logger.info(f"[Orchestrator] 第二次 LLM 完成原因: '{second_llm_response.choices[0].finish_reason}'")
                     raw_final_content = final_response_message.content
                     # 记录最终响应的原始内容（截断）
                     logger.debug(f"[Orchestrator] 第二次 LLM 原始 Content (前 1000 字): >>>\n{str(raw_final_content)[:1000]}...\n<<<")

                     # 使用 OutputParser 解析最终的 <think> 和 回复文本
                     final_thinking, final_reply = self.output_parser._parse_llm_text_content(raw_final_content)

                     # 将最终的 assistant 回复（原始消息对象）添加到短期记忆
                     try:
                         final_assistant_message_dump = final_response_message.model_dump(exclude_unset=True)
                         self.memory_manager.add_to_short_term(final_assistant_message_dump)
                         logger.debug("[Orchestrator] 最终 LLM 回复 (原始 Message Dump) 已添加至短期记忆。")
                     except Exception as mem_err:
                         logger.error(f"添加最终回复到短期记忆失败: {mem_err}", exc_info=True)

                 # 组合最终返回给用户的报告字符串
                 final_report = f"<think>{final_thinking}</think>\n\n{final_reply}".rstrip()

            except Exception as e:
                 # 捕获第二次 LLM 调用或最终报告处理过程中的所有其他错误
                 logger.critical(f"[Orchestrator] 第二次 LLM 调用或最终报告处理失败: {e}", exc_info=True)
                 # 生成一个备用的错误报告
                 fallback_thinking = f"第二次 LLM 调用或最终报告处理失败: {e}"
                 fallback_reply = f"抱歉，在为您准备最终报告时遇到了严重的内部错误 ({e})！请参考日志获取技术详情。"
                 # 尝试记录错误到记忆
                 try: self.memory_manager.add_to_short_term({"role": "assistant", "content": f"<think>{fallback_thinking}</think>\n\n{fallback_reply}"})
                 except Exception: pass
                 final_report = f"<think>{fallback_thinking}</think>\n\n{fallback_reply}".rstrip()
                 await async_print(f"\n🔴 抱歉，生成最终报告时发生严重错误 ({e})！")

            # 请求处理结束（工具调用路径）
            request_end_time = time.monotonic()
            logger.info(f"\n{'='*25} V7.1 请求处理完毕 (工具调用路径, 耗时: {request_end_time - request_start_time:.3f} 秒) {'='*25}\n")
            return final_report

        else: # is_tool_calls is False
            # --- 情况 B: 计划不需要执行工具，直接回复 ---
            logger.info("[Orchestrator] 决策：根据 JSON 计划直接回复，不执行工具。")
            await async_print("--- 大脑认为无需执行操作，将直接回复... ---")

            # 我直接使用第一次 LLM 调用（规划阶段）生成的 'direct_reply'
            if direct_reply_from_plan and isinstance(direct_reply_from_plan, str) and direct_reply_from_plan.strip():
                # 使用规划中提供的回复
                logger.info("[Orchestrator] 使用计划中提供的 'direct_reply' 作为最终回复。")
                final_thinking = thinking_process # 复用第一次 LLM 的思考过程
                final_reply = direct_reply_from_plan
                # 在这种情况下，没有第二次 LLM 调用
            else:
                # 如果 is_tool_calls 为 false，但 direct_reply 无效或缺失，这是一个规划错误
                logger.error("[Orchestrator] 规划错误: 'is_tool_calls' 为 false 但 'direct_reply' 无效或缺失！")
                await async_print("\n🔴 内部错误：计划无需操作，但大脑没有提供回复内容！")
                # 生成一个包含错误信息的回复
                final_thinking = thinking_process + "\n规划错误：is_tool_calls 为 false 但 direct_reply 无效或缺失。"
                final_reply = "我理解现在不需要执行操作，但是智能大脑没有提供相应的回复。这可能是一个规划错误，请您澄清指令或重试。"

            # 组合最终报告
            final_report = f"<think>{final_thinking}</think>\n\n{final_reply}".rstrip()
            # 注意：第一次 LLM 的规划响应（包含 direct_reply）已经在本函数前面添加到记忆中了

            # 请求处理结束（直接回复路径）
            request_end_time = time.monotonic()
            logger.info(f"\n{'='*25} V7.1 请求处理完毕 (直接回复路径, 耗时: {request_end_time - request_start_time:.3f} 秒) {'='*25}\n")
            return final_report

    # --- Helper Methods for Prompts ---
    def _get_tool_schemas_for_prompt(self) -> str:
        """
        根据 `self.tools_registry` 中的信息动态生成工具描述字符串，用于注入 LLM Prompt。
        这样我就不必在 Prompt 中硬编码工具列表了。
        """
        if not self.tools_registry:
            # 如果没有注册任何工具，明确告知 LLM
            return "  (无可用工具)"

        tool_schemas = []
        # 遍历注册表中的每个工具
        for name, schema in self.tools_registry.items():
            desc = schema.get('description', '无描述。') # 获取描述
            params = schema.get('parameters', {}) # 获取参数 Schema
            props = params.get('properties', {}) # 获取具体参数属性
            req = params.get('required', []) # 获取必需参数列表
            param_desc_parts = []
            if props:
                # 格式化每个参数的描述
                for k, v in props.items():
                    p_type = v.get('type', 'any') # 参数类型
                    p_desc = v.get('description', '') # 参数描述
                    # 标记参数是否必需
                    p_req = '(必须)' if k in req else '(可选)'
                    param_desc_parts.append(f"{k}: {p_type} {p_req} '{p_desc}'")
                param_desc_str = "; ".join(param_desc_parts) # 用分号连接各参数描述
            else:
                param_desc_str = "无参数" # 如果没有参数
            # 格式化单个工具的完整描述
            tool_schemas.append(f"  - `{name}`: {desc} (参数: {param_desc_str})")
        # 将所有工具的描述合并为一个字符串
        return "\n".join(tool_schemas)

    def _get_planning_prompt_v7(self, tool_schemas_desc: str, memory_context: str) -> str:
        """
        构建第一次规划调用的 System Prompt。
        核心要求是 LLM 严格按照 `<think>...</think> JSON_OBJECT` 格式输出，
        其中 JSON 对象描述了执行计划（调用哪些工具，或直接回复）。
        我明确禁止 LLM 使用其内置的 Function Calling 功能。
        """
        return (
            "你是一位顶尖的、极其严谨的电路设计编程助理。你的行为必须专业、精确，并严格遵循指令。\n"
            "你的任务是：分析用户的最新指令、对话历史以及当前的电路状态，然后**严格按照下面描述的固定格式**生成一个包含执行计划的 JSON 对象。\n"
            "**绝对禁止**使用任何形式的 Function Calling 或生成 `tool_calls` 字段。你的**唯一输出**必须由两部分组成：\n"
            "1.  一个 `<think>...</think>` XML 块：在其中详细阐述你的思考过程。这应包括：对用户指令的理解，对当前电路状态和记忆的分析，决定是否需要调用工具以及调用哪些工具，如何从指令中提取参数，规划具体的执行步骤，以及对潜在问题的评估。\n"
            "2.  **紧随其后**，**必须**是一个**单一的、格式完全正确的 JSON 对象**：这个 JSON 对象代表了你最终的执行计划或直接回复。**不允许在 JSON 对象的前面或后面添加任何额外的文字、解释或注释！**\n\n"
            "**JSON 对象格式规范 (必须严格遵守):**\n"
            "该 JSON 对象**必须**包含以下字段：\n"
            "  - `is_tool_calls` (boolean): **必须字段**。如果分析后认为需要执行一个或多个工具操作来满足用户请求，则设为 `true`。如果不需要执行任何工具（例如，可以直接回答问题、进行确认、或者认为请求无法处理），则设为 `false`。\n"
            "  - `tool_list` (array<object> | null): **必须字段**。行为取决于 `is_tool_calls` 的值：\n"
            "     - 当 `is_tool_calls` 为 `true` 时: **必须**是一个包含**一个或多个**工具调用对象的数组。数组中的对象**必须按照你期望的执行顺序列出**。\n"
            "     - 当 `is_tool_calls` 为 `false` 时: 此字段**必须**是 `null` 值或者一个空数组 `[]`。\n"
            "     每个工具调用对象（如果存在）**必须**包含以下字段：\n"
            "       - `toolname` (string): **必须**。要调用的工具的**精确名称**。你必须从下面提供的“可用工具列表”中选择一个有效的名称。\n"
            "       - `params` (object): **必须**。一个包含调用该工具所需参数的 JSON 对象。你必须**严格**根据该工具的参数规范，从用户指令或对话历史中提取参数值。如果某个工具不需要参数，则提供一个空对象 `{}`。\n"
            "       - `index` (integer): **必须**。表示此工具调用在你本次规划中的执行顺序。**必须**是一个从 `1` 开始的正整数。如果本次规划包含多个工具调用，它们的 `index` 值必须是连续的（例如 1, 2, 3）。\n"
            "  - `direct_reply` (string | null): **必须字段**。行为取决于 `is_tool_calls` 的值：\n"
            "     - 当 `is_tool_calls` 为 `false` 时: 这里**必须**包含你准备直接回复给用户的最终、完整、友好的文本内容。回复内容不能为空字符串。\n"
            "     - 当 `is_tool_calls` 为 `true` 时: 此字段**必须**是 `null` 值。（因为后续会通过执行工具并再次调用你来生成最终回复）。\n\n"
            "**可用工具列表与参数规范:**\n"
            # 工具描述是动态生成的
            f"{tool_schemas_desc}\n\n"
            "**当前电路状态与记忆:**\n"
            # 注入当前的电路状态和近期记忆
            f"{memory_context}\n\n"
            "**最后再次强调：你的回复格式必须严格是 `<think>思考过程</think>` 后面紧跟着一个符合上述规范的 JSON 对象。不允许有任何偏差！** JSON 的语法（括号、引号、逗号、数据类型）和结构（必需字段、条件字段）都必须完全正确，否则后续处理会失败。"
        )

    def _get_response_generation_prompt_v7(self, memory_context: str, tool_schemas_desc: str, tools_were_skipped: bool) -> str:
        """
        构建第二次响应生成调用的 System Prompt。
        此时，LLM 已经看到了用户请求、它的规划、以及所有已执行工具的结果（在 'tool' 消息中）。
        这个 Prompt 的核心任务是要求 LLM 基于所有这些信息，生成一个最终的、面向用户的回复。
        我特别强调了 LLM 需要理解 'tool' 消息中的 `status` 字段，并能在报告中反映工具执行的成功与失败，
        以及解释为何某些计划中的步骤（如果 `tools_were_skipped` 为 true）没有执行。
        """
        # 根据是否有工具被跳过，动态添加提示信息
        skipped_info = ""
        if tools_were_skipped:
            skipped_info = "\n**重要提示：** 在之前的工具执行过程中，由于某个工具失败，后续计划中的一个或多个工具可能已被跳过执行。请在你的最终报告中明确说明这一点，解释哪些任务（如果有的话）因此未能完成，并总结当前任务的最终状态。"

        return (
            "你是一位顶尖的电路设计编程助理，经验丰富，技术精湛，并且擅长清晰地汇报工作结果。\n"
            "你的当前任务是：基于到目前为止的完整对话历史（包括用户最初的指令、你之前的思考和规划、以及所有已执行工具的结果），生成最终的、面向用户的文本回复。\n"
            "**关键信息来源是角色为 'tool' 的消息**: 每条 'tool' 消息都对应一个之前执行的工具调用（通过 `tool_call_id` 关联）。其 `content` 字段是一个 JSON 字符串，包含了该工具执行的关键信息，特别是 `status` 字段（指示 'success' 或 'failure'）和 `message` 字段（描述结果或错误）。可能还包含 `error` 字段提供失败的详细技术信息。\n"
            "**你的报告必须：**\n"
            "1.  仔细阅读并理解所有历史消息，**特别是要解析每条 'tool' 消息中的 JSON 内容**，准确把握每个已执行工具的**最终状态 (`status`)** 和结果 (`message`)。\n"
            "2.  清晰地向用户总结所有**已执行**工具操作的结果：\n"
            "    - 对于成功的操作 (`status: \"success\"`)，进行简要的确认。\n"
            "    - 对于失败的操作 (`status: \"failure\"`)，必须诚恳地向用户说明操作失败了，并根据 `message` 和可能的 `error` 字段解释失败的原因及其对整体任务的影响。\n"
            f"{skipped_info}\n" # 如果有工具被跳过，在这里插入提示
            "3.  综合以上信息，回答用户最初的问题或确认任务的完成情况。如果任务因工具失败而未能完全完成，请明确说明当前的状态。\n"
            "4.  **严格按照以下固定格式**生成你的回复：\n"
            "   a. **思考过程:** 首先，在 `<think>` 和 `</think>` 标签之间，详细阐述你的反思和报告组织思路。回顾用户的原始请求、你的规划、并**重点分析所有已执行工具的 `status` 和 `message`**。评估任务的整体完成度。如果 `tools_were_skipped` 为真，要考虑未执行步骤的影响。最后，规划如何将这些信息整合，组织成清晰、友好的最终回复。\n"
            "   b. **正式回复:** 在 `</think>` 标签之后，紧跟着面向用户的正式文本回复。这个回复应该基于你的思考过程，清晰、简洁、友好地总结情况。\n"
            "**最终输出格式必须严格是:**\n"
            "`<think>你的详细思考过程</think>\n\n你的正式回复文本`\n"
            "(注意：`</think>` 标签后必须恰好是两个换行符 `\\n\\n`，然后直接是正式回复文本。)\n"
            "**重要：** 在这个阶段，你**绝对不能**再生成任何工具调用或 JSON 对象。你的唯一输出应该是包含 `<think>` 块和正式回复文本的字符串。"
            "\n\n"
            "**上下文参考信息:**\n"
            "【当前电路状态与记忆】\n"
            f"{memory_context}\n"
            "【我的可用工具列表 (仅供你参考，不应再次调用)】\n"
            f"{tool_schemas_desc}\n"
        )


# --- 异步主函数 (应用程序入口) ---
async def main():
    """
    异步主函数。我负责初始化 Agent 并启动主交互循环，处理用户输入。
    """
    # 使用我定义的异步打印函数来输出启动信息
    await async_print("=" * 70)
    await async_print("🚀 启动 OpenManus 电路设计 Agent (V7.1) 🚀")
    await async_print("   特性: 异步核心, 动态工具注册, 规划重试, 失败中止, 内存修剪")
    await async_print("=" * 70)
    logger.info("[Main] 开始 Agent 初始化...")

    # --- 获取 API Key ---
    # 我优先尝试从环境变量 `ZHIPUAI_API_KEY` 读取 API Key
    api_key = os.environ.get("ZHIPUAI_API_KEY")
    if not api_key:
        # 如果环境变量未设置，我提示用户手动输入
        logger.warning("[Main] 环境变量 ZHIPUAI_API_KEY 未设置。")
        await async_print("\n为了连接智能大脑，我需要您的智谱AI API Key。")
        try:
            # 注意：标准 input() 是同步阻塞调用。在简单的命令行应用中可以接受。
            # 对于需要更高并发或非阻塞输入的场景（如 GUI, Web），应使用相应的异步输入库。
            api_key = input("👉 请在此输入您的智谱AI API Key: ").strip()
        except (EOFError, KeyboardInterrupt):
            # 处理用户中断输入的情况
            await async_print("\n输入被中断。程序即将退出。")
            logger.info("[Main] 用户中断了 API Key 输入。")
            return # 退出 main 函数
        # 再次检查用户是否输入了 Key
        if not api_key:
            await async_print("\n错误：未提供 API Key。程序无法启动，即将退出。")
            logger.critical("[Main] 用户未提供 API Key。")
            return
        logger.info("[Main] 已通过手动输入获取 API Key。")
        await async_print("API Key 已获取。")

    # --- 初始化 Agent ---
    agent = None
    try:
        # 实例化 Agent 核心类
        # 我在这里可以配置 Agent 的参数，例如使用的 LLM 模型、重试次数、记忆容量等
        agent = CircuitDesignAgentV7(
            api_key=api_key,
            model_name="glm-4-flash", # 可以更换为其他兼容模型，如 glm-4
            planning_llm_retries=1,   # 设置规划失败时的重试次数
            max_short_term_items=25   # 设置短期记忆的最大条目数
        )
        await async_print("\n🎉 Agent V7.1 初始化成功！已准备就绪。")
        # 提供一些示例指令给用户参考
        await async_print("\n您可以尝试以下指令:")
        await async_print("  - '给我加个1k电阻R1K和3V电池B3V'")
        await async_print("  - '连接R1K和B3V'")
        await async_print("  - '电路现在什么样？'")
        await async_print("  - '尝试连接 R1K 和一个不存在的元件 XYZ'") # 测试工具失败和提前中止
        await async_print("  - '清空电路'")
        await async_print("  - '你好，今天天气怎么样？'") # 测试不需要工具的直接回复
        await async_print("  - 输入 '退出' 来结束程序")
        await async_print("-" * 70)
    except Exception as e:
        # Agent 初始化失败是严重错误，程序无法继续
        logger.critical(f"[Main] Agent V7.1 初始化失败: {e}", exc_info=True)
        await async_print(f"\n🔴 Agent 初始化失败！错误: {e}。请检查日志和配置。程序即将退出。")
        return # 从 main 函数返回，程序将结束

    # --- 主交互循环 ---
    # 我在这里循环接收用户输入，并调用 Agent 处理请求
    try:
        while True:
            try:
                user_input = ""
                try:
                    # 获取用户输入。同样，这是同步阻塞调用。
                    user_input = input("用户 > ").strip()
                except (EOFError, KeyboardInterrupt):
                    # 处理用户输入中断 (Ctrl+D 或 Ctrl+C)
                    raise # 重新抛出，由外层 try...except 捕获并处理退出

                # 检查是否是退出指令
                if user_input.lower() in ['退出', 'quit', 'exit', '再见', '结束', 'bye']:
                    await async_print("\n收到退出指令。感谢您的使用！👋")
                    logger.info("[Main] 收到退出指令，结束交互循环。")
                    break # 跳出 while 循环

                # 如果用户只输入了空白符，则忽略并继续等待下一条指令
                if not user_input:
                    continue

                # **调用 Agent 的核心处理方法** (这是一个异步调用)
                start_process_time = time.monotonic() # 记录处理开始时间
                response = await agent.process_user_request(user_input)
                process_duration = time.monotonic() - start_process_time # 计算处理耗时

                # 打印 Agent 返回的完整响应（通常包含 <think> 和正式回复）
                await async_print(f"\n📝 Agent 回复 (耗时: {process_duration:.3f} 秒):")
                await async_print(response) # Agent 返回的是一个字符串
                await async_print("-" * 70) # 分隔符，使交互更清晰

            except KeyboardInterrupt:
                # 如果在 Agent 处理请求期间（await 时）用户按 Ctrl+C
                await async_print("\n用户操作被中断。")
                logger.info("[Main] 用户中断了当前请求的处理。")
                # 这里选择退出整个循环
                break
            except EOFError:
                # 如果输入流意外结束 (例如在管道或重定向输入时)
                await async_print("\n输入流意外结束。")
                logger.info("[Main] 输入流结束 (EOF)。")
                break
            except Exception as loop_err:
                # 捕获 Agent 处理请求过程中未被内部捕获的意外错误
                # 这有助于防止整个交互循环因单次请求失败而崩溃
                logger.error(f"[Main] 处理指令 '{user_input[:50]}...' 时发生意外错误: {loop_err}", exc_info=True)
                await async_print(f"\n🔴 Agent 运行时错误:")
                # 我生成一个安全的错误报告给用户，避免再次调用可能出错的 Agent 逻辑
                error_report = f"<think>处理指令 '{user_input[:50]}...' 时发生内部错误: {loop_err}\n{traceback.format_exc()}</think>\n\n抱歉，我在执行您的指令时遇到了意外问题 ({loop_err})！我已经记录了详细的技术信息。请尝试其他指令或检查日志。"
                await async_print(error_report)
                await async_print("-" * 70)
                # 发生错误后，继续下一次循环，尝试处理用户的下一条指令
                continue

    except Exception as outer_loop_err:
        # 捕获主交互循环本身之外的、更严重的未处理异常
        logger.critical(f"[Main] 主交互循环外发生未处理异常: {outer_loop_err}", exc_info=True)
        await async_print(f"\n🔴 严重系统错误导致交互循环终止: {outer_loop_err}。")
    finally:
        # 无论循环如何结束（正常退出、异常退出），都执行这里的清理代码
        logger.info("[Main] 主交互循环结束。")
        await async_print("\n正在关闭 Agent V7.1...")
        # 如果 Agent 类需要执行任何清理操作（例如关闭网络连接、保存状态），
        # 可以在这里调用相应的方法，例如: await agent.cleanup()

# --- 用于 Jupyter/IPython 环境的辅助函数 ---
# 在 Jupyter Notebook 或类似环境中，直接运行 `asyncio.run(main())` 可能与现有的事件循环冲突。
# 这个函数提供了一种在这些环境中启动 Agent 交互的方式。
async def run_agent_in_jupyter():
    """
    在 Jupyter/IPython 环境中安全启动 Agent 交互循环的辅助函数。
    你应该在 Notebook cell 中使用 `await run_agent_in_jupyter()` 来调用它。
    """
    print("正在尝试以 Jupyter/IPython 兼容模式启动 Agent V7.1...")
    print("请在下方的输入提示处输入指令。输入 '退出' 结束。")
    # 直接调用并等待 main 协程完成
    try:
        await main()
    except Exception as e:
        # 捕获在 Jupyter 中运行时可能发生的异常
        print(f"\n🔴 Agent 在 Jupyter 模式下运行时遇到错误: {e}")
        logger.error(f"在 Jupyter 模式下运行 Agent 时出错: {e}", exc_info=True)
    finally:
        print("Agent 交互已结束 (Jupyter 模式)。")


# # --- 标准 Python 脚本入口点 ---
# if __name__ == "__main__":
#     # 我首先判断当前是否运行在 IPython/Jupyter 环境中
#     # 这是因为在这些环境中启动 asyncio 事件循环的方式可能与标准脚本不同
#     try:
#         # `get_ipython` 是 IPython 环境提供的内置函数
#         shell = get_ipython().__class__.__name__
#         # 'ZMQInteractiveShell' 是 Jupyter Notebook 和 QtConsole 使用的 shell 类型
#         if shell == 'ZMQInteractiveShell':
#             # 如果是在 Jupyter Notebook/Lab 或 QtConsole 中
#             print("检测到 Jupyter/IPython (ZMQ) 环境。")
#             print("请在 Notebook cell 中执行 `await run_agent_in_jupyter()` 来启动 Agent 交互。")
#             # 在这种情况下，我不自动启动 `main`，而是提示用户手动启动
#         elif shell == 'TerminalInteractiveShell':
#             # 如果是在终端中使用 `ipython` 命令启动的交互式 shell
#             print("检测到终端 IPython 环境。将尝试作为标准脚本启动...")
#             raise RuntimeError("Terminal IPython detected, run as script.") # 抛出异常以进入下面的 except 块
#         else:
#             # 其他类型的 IPython shell (比较少见)
#             print(f"检测到非典型的 IPython shell ({shell})。将尝试作为标准脚本启动...")
#             raise RuntimeError("Other IPython shell detected, run as script.")
#     except (NameError, RuntimeError):
#         # 如果 `get_ipython` 未定义 (即在标准 Python 解释器中运行)
#         # 或者从上面的 `RuntimeError` 进入此块
#         print("正在以标准 Python 脚本模式启动 Agent V7.1...")
#         try:
#             # 使用 `asyncio.run()` 来运行异步的 `main` 函数
#             # 这是在标准 .py 文件中运行顶层异步代码的推荐方式
#             asyncio.run(main())
#         except KeyboardInterrupt:
#             # 捕获在 `asyncio.run()` 运行期间用户按下的 Ctrl+C
#             print("\n程序被用户强制退出 (KeyboardInterrupt)。")
#             logger.info("[Main Script] 程序被 KeyboardInterrupt 中断。")
#         except Exception as e:
#             # 捕获在 `asyncio.run()` 执行期间可能发生的其他顶层异常
#             print(f"\n程序因顶层错误而意外退出: {e}")
#             logger.critical(f"脚本执行期间发生顶层异常: {e}", exc_info=True)
#         finally:
#             # 无论如何退出，都打印结束信息
#             print("Agent V7.1 程序已关闭。")

In [ ]:
await main()

# OpenManus V7.0 - 异步 & 增强基础版 - 超详细文档

## 1. 概述 / 介绍

OpenManus V7.0 是对我们电路设计智能助理的一次**重大架构升级**。在 V6.2 的基础上，我们聚焦于解决几个关键痛点：用户交互时的**阻塞感**、LLM 输出不稳定导致的**规划脆弱性**、以及长期运行可能导致的**上下文无限增长**问题。

**V7.0 的核心目标与改进:**

*   **提升响应性 (Asynchronous Core):** 通过引入 Python 的 `asyncio` 库，将核心 Agent 处理流程 (`process_user_request`) 以及涉及等待的操作（如 LLM API 调用、工具执行的协调）改造为异步模式。这意味着在等待 LLM 回复或工具执行时，程序不会完全“卡死”，理论上可以处理其他任务（在当前单用户脚本中，主要体现为非阻塞等待），从而**显著降低用户感知的单次交互总时长**。
*   **增强规划鲁棒性 (Planning Retry & Custom JSON):**
    *   **自定义 JSON 规划:** V7 强制要求 LLM 在第一阶段生成一个结构**非常严格**的自定义 JSON 对象作为执行计划，而不是依赖 LLM API 可能不稳定的原生 Function Calling/Tool Usage。这使我们能进行精确的**事前解析和验证**。
    *   **解析重试:** 针对 LLM 可能偶尔不完全遵守 JSON 格式的问题，增加了在解析此自定义 JSON 失败时的**自动重试机制**（默认1次），提高了从 LLM 获取有效规划的成功率。
*   **优化执行控制 (Tool Failure Handling - Early Exit):** `ToolExecutor` 现在会在**每个**工具成功或失败后检查其返回状态。一旦有工具执行失败，将**立即停止**执行后续计划中的所有工具。这避免了基于错误状态继续执行导致的潜在问题，并将失败信息准确地传递给第二阶段的 LLM 进行总结。
*   **基础内存管理 (Memory Pruning):** `MemoryManager` 增加了**基于消息数量的短期记忆修剪**机制。当对话历史超过预设长度 (`max_short_term_items`) 时，会自动移除最旧的非系统消息，防止上下文无限膨胀，确保请求能在 LLM 的 Token 限制内。
*   **标准化接口与错误处理 (Standardized Errors):** 规定了所有 Action 方法（工具实现）必须返回包含 `status` ("success" 或 "failure") 和 `message` 的标准字典格式。`ToolExecutor` 对此进行检查，日志记录和错误传递更加规范。

V7.0 旨在构建一个**更健壮、更高效、更能适应复杂交互**的 Agent 基础，为未来添加更高级的功能（如 RAG、复杂错误恢复策略等）打下坚实的地基。

## 2. 实现思路 (设计哲学深度解析)

V7.0 的设计哲学是**控制、健壮性与异步效率**的结合，核心思路基于对标准 Agent 模式（如 ReAct）的改进：

1.  **两阶段 LLM 交互 (解耦规划与报告):**
    *   **为什么分两步?** 将复杂的任务分解。第一步 LLM 专注于**理解意图、检索状态并制定精确计划** (输出严格的 JSON)。第二步 LLM 则专注于**综合执行结果、处理成功与失败、并生成用户友好的报告**。这种解耦降低了单次 LLM 调用的认知负担，提高了各自任务的成功率。
    *   **第一阶段 (规划 - Custom JSON):**
        *   **核心:** 输出一个自定义结构的 JSON。
        *   **优势:**
            *   **显式控制:** 我们可以精确定义计划的结构，强制 LLM 按我们的要求思考和输出。
            *   **强验证:** 在执行任何操作前，可以对 JSON 进行严格的语法和逻辑验证。
            *   **LLM 专注:** 让 LLM 集中精力生成结构化数据，而不是混合代码执行逻辑。
        *   **劣势:**
            *   **Prompt 工程复杂:** 需要精心设计 System Prompt，明确指示 LLM 输出此特定 JSON 格式，并处理其可能不遵从的情况。
            *   **LLM 遵从性风险:** LLM 可能仍然无法完美生成所需 JSON，需要解析器有足够的鲁棒性并配合重试机制。
    *   **第二阶段 (报告 - Synthesis):**
        *   **核心:** 输入包括原始指令、第一阶段计划、所有工具执行结果（含状态），输出自然语言报告。
        *   **优势:** 让 LLM 发挥其强大的文本生成和总结能力，将技术性的执行日志转化为易于理解的回复，并能根据工具的成功/失败状态调整措辞。

2.  **异步编排 (`asyncio`):**
    *   **目的:** 解决 I/O 阻塞问题。LLM API 调用是典型的网络 I/O 操作，等待时间可能较长。
    *   **机制:** 使用 `async`/`await` 关键字。当遇到 `await` 等待一个异步操作（如 `await llm_interface.call_llm(...)` 或 `await tool_executor.execute_tool_calls(...)` 中的 `await asyncio.to_thread(...)`）完成时，事件循环会将控制权交还，允许程序处理其他已就绪的任务（或在单任务脚本中，只是非阻塞地等待）。这使得程序在等待期间保持“活跃”。

3.  **受控且安全的执行 (Validation & Early Exit):**
    *   **事前验证:** 在执行任何工具前，严格验证 LLM 规划的 JSON 结构。
    *   **事后检查 + 提前退出:** `ToolExecutor` 不仅执行，还**监控**每个工具的返回值 (`status`)。失败则**立即停止**后续工具的执行。
        *   **意义:** 防止在一个失败的操作（可能导致状态不一致）基础上继续执行后续依赖步骤，提高了整体流程的安全性和可预测性。失败被视为当前轮次规划执行的中止点。

4.  **模块化设计 (Componentization):**
    *   **目的:** 将不同职责的代码分离到独立的类中（`MemoryManager`, `LLMInterface`, `OutputParser`, `ToolExecutor`）。
    *   **优势:**
        *   **高内聚、低耦合:** 每个类专注于自己的任务。
        *   **易于维护和升级:** 修改一个组件的内部实现（例如更换 LLM API 或改进解析逻辑）对其他组件的影响较小。
        *   **可测试性:** 可以独立测试每个组件的功能。

5.  **显式状态管理 (`MemoryManager`):**
    *   **职责:** 作为 Agent 状态的唯一真实来源 (Single Source of Truth)。管理三种主要状态：
        *   **对话历史 (Short-Term):** 用户与 Agent 之间的消息流，用于 LLM 理解上下文。
        *   **电路知识库 (Structured State):** 关于当前电路设计的结构化数据（元件字典、连接集合），供 Action 方法直接读写和 LLM 参考。
        *   **长期经验 (Long-Term Placeholder):** 简单的知识片段列表，代表未来 RAG 能力的基础。
    *   **修剪目的:** 控制传递给 LLM 的上下文长度，防止超出模型限制并降低 API 成本。V7 使用的是基础的基于数量的 FIFO（先进先出，排除系统消息）策略。

## 3. 核心组件 (深入解析)

---

### 3.1. `CircuitDesignAgentV7` (大脑中枢与协调器)

*   **角色:** 整个 Agent 的“指挥官”，负责初始化并协调所有其他组件，驱动交互流程。
*   **核心职责:**
    1.  **持有组件实例:** 在 `__init__` 中创建并持有 `MemoryManager`, `LLMInterface`, `OutputParser`, `ToolExecutor` 的实例。
    2.  **定义工具 Schema:** `self.tools` 列表定义了 Agent *能做什么*，包括工具名称、描述、参数。**注意：此 Schema 在 V7 中仅用于生成提示信息告知 LLM 有哪些可用工具，并不直接用于 ZhipuAI SDK 的 `tools` 参数。**
    3.  **实现主循环 (`process_user_request`):** 这是核心业务逻辑所在，编排了从接收用户输入到返回最终响应的完整异步流程（详见交互流程部分）。
    4.  **生成动态提示:** 通过 `_get_planning_prompt_v7` 和 `_get_response_generation_prompt_v7` 等辅助方法，根据当前上下文（记忆、工具）动态构建传递给 LLM 的 System Prompt。
    5.  **托管 Action 方法:** 实际执行工具功能的代码（如 `add_component_tool`）作为 Agent 类的方法存在，由 `ToolExecutor` 回调。
*   **关键方法:**
    *   `__init__(...)`: 初始化所有依赖组件，加载配置（API Key, 模型名等），定义工具列表。包含重要的初始化错误处理。
    *   `async process_user_request(user_request: str) -> str`: 异步处理单次用户交互的主入口。包含了完整的 感知->规划->行动->观察->报告 循环。
    *   `_get_..._prompt_v7(...)`: 内部方法，用于生成结构化、包含实时上下文的系统提示。
    *   **Action 方法** (e.g., `add_component_tool`): 详见下方 Action 方法部分。

---

### 3.2. `LLMInterface` (LLM API 网关)

*   **角色:** 封装与 ZhipuAI 大语言模型 API 的所有通信细节。
*   **核心职责:**
    1.  **初始化客户端:** 使用 API Key 配置并创建 `zhipuai.ZhipuAI` 客户端实例。
    2.  **执行异步 API 调用 (`async call_llm`):**
        *   接收 `messages` 列表和其他参数（模型名、温度等）。
        *   **关键：** 使用 `asyncio.to_thread(self.client.chat.completions.create, ...)` 来执行同步的 SDK 调用。这是因为 `zhipuai` SDK (截至代码编写时，其常用 `create` 方法是同步阻塞的) 直接在异步函数中调用会阻塞事件循环。`asyncio.to_thread` 将此阻塞调用移到后台线程执行，使 `call_llm` 方法本身成为非阻塞的异步函数。
        *   处理 API 返回：提取主要内容、记录 token 使用量、完成原因。
        *   **错误处理:** 捕获 API 调用过程中可能发生的网络错误、认证错误等 (`zhipuai` SDK 可能抛出的异常），并向上抛出。
        *   **V7 特定:** 在规划阶段，`use_tools` 参数应为 `False`，因为 V7 依赖自定义 JSON 而不是 SDK 的工具调用机制。
*   **关键方法:**
    *   `__init__(...)`: 设置 API Key, 模型名, 默认参数，初始化 `ZhipuAI` 客户端。
    *   `async call_llm(...)`: 封装异步 API 请求逻辑，包括使用 `asyncio.to_thread`、参数传递、基本响应处理和错误捕获。

---

### 3.3. `MemoryManager` (Agent 的记忆库)

*   **角色:** 存储和管理 Agent 的状态信息。
*   **核心职责:**
    1.  **维护短期记忆 (`short_term`):** 存储对话消息列表（用户输入 `user`, Agent 回复 `assistant`, 工具结果 `tool`）。列表项是符合 OpenAI 格式的字典。
    2.  **应用短期记忆修剪:** 在 `add_to_short_term` 中，检查当前消息数是否超过 `max_short_term_items`。如果超过，则移除列表中第一个非 `role: system` 的消息（通常是最旧的用户或助手消息），以保持列表长度。
    3.  **维护长期记忆占位符 (`long_term`):** 一个简单的字符串列表，用于存储一些总结性的知识片段。V7 中仅追加，并在达到 `max_long_term_items` 时移除最旧的。**这是未来实现 RAG 的基础，目前功能非常有限。**
    4.  **维护电路状态 (`circuit_knowledge`):** 使用字典 (`components`) 存储 `CircuitComponent` 对象，键是元件 ID；使用集合 (`connections`) 存储表示连接的元组（排序以保证唯一性）；使用字典 (`_component_counters`) 跟踪各种类型元件的 ID 计数器。这是 Agent 对当前设计任务状态的**结构化理解**。
    5.  **提供上下文信息:** `get_circuit_state_description` 生成电路的文本描述。`get_memory_context_for_prompt` 组合电路描述和**最近 N 条**长期记忆（明确提示了其局限性）为 LLM 提供背景。
    6.  **管理元件 ID:** `generate_component_id` 根据元件类型（及其映射代码）和计数器生成唯一的 ID，并处理潜在的冲突。
*   **关键方法:**
    *   `add_to_short_term(...)`: 添加消息并执行修剪。
    *   `add_to_long_term(...)`: 添加知识片段并执行修剪。
    *   `get_circuit_state_description()`: 返回电路状态文本。
    *   `get_memory_context_for_prompt(...)`: 返回格式化的上下文（电路+近期长期记忆）。
    *   `generate_component_id(...)`: 生成唯一元件 ID。
    *   `clear_circuit()`: 清空电路知识和计数器。

---

### 3.4. `OutputParser` (LLM 响应解码器)

*   **角色:** 负责从 LLM 返回的原始文本中提取结构化信息（自定义 JSON）和非结构化信息（思考过程、回复文本）。
*   **核心职责:**
    1.  **解析规划响应 (`parse_planning_response`):**
        *   **输入:** LLM 返回的消息对象 (可能为 None)。
        *   **输出:** 元组 `(thinking_process: str, plan: Optional[Dict], error_message: str)`。
        *   **步骤:**
            *   检查输入是否有效。
            *   使用正则表达式 `re.search(r'<think>(.*?)</think>...)` 提取 `<think>` 标签内的思考过程。
            *   **鲁棒地定位 JSON:** 从 `<think>` 标签结束后开始，或者如果没有 `<think>`，从第一个 `{` 或 `[` 开始，智能地找到匹配的结束 `}` 或 `]`，提取出 JSON 字符串，即使 LLM 在 JSON 前后添加了额外文本。
            *   清理 JSON 字符串（移除 ```json, ``` 等标记）。
            *   使用 `json.loads()` 解析提取出的字符串。
            *   **严格验证:** 检查解析结果是否为字典，是否包含必需的键 (`is_tool_calls`, `tool_list`, `direct_reply`)，以及这些键的类型和值是否符合 V7 的规范（例如，`is_tool_calls` 为 `true` 时 `tool_list` 不能为空列表，`index` 必须是连续正整数等）。
            *   **错误处理:** 捕获 `json.JSONDecodeError` (格式错误) 和 `ValueError` (结构验证失败)，生成详细的错误信息。
    2.  **解析最终报告 (`_parse_llm_text_content`):**
        *   **输入:** 第二次 LLM 调用返回的 `content` 字符串。
        *   **输出:** 元组 `(thinking_process: str, formal_reply: str)`。
        *   **步骤:**
            *   使用正则表达式提取 `<think>` 内容。
            *   将 `<think>` 标签之前和之后的内容（去除空白）分别处理或合并，作为正式回复。如果未找到 `<think>` 标签，则整个内容视为回复。
*   **关键方法:**
    *   `parse_planning_response(...)`: 处理第一次 LLM 调用，提取思考和自定义 JSON 计划，并进行严格验证。
    *   `_parse_llm_text_content(...)`: 处理第二次 LLM 调用，提取思考和最终的自然语言回复。

---

### 3.5. `ToolExecutor` (Action 执行引擎)

*   **角色:** 负责按顺序、安全地执行 Agent 的 Action 方法（工具）。
*   **核心职责:**
    1.  **接收模拟工具调用列表:** 输入是由 `CircuitDesignAgentV7` 从验证通过的自定义 JSON 计划转换而来的“模拟”工具调用字典列表。
    2.  **按顺序迭代执行:** 严格按照列表顺序处理每个模拟调用。
    3.  **参数解析与准备:** 从模拟调用的 `function['arguments']` (JSON 字符串) 解析出参数字典。
    4.  **查找并调用 Action:** 使用 `getattr(self.agent_instance, function_name)` 动态查找 `CircuitDesignAgentV7` 实例上对应的 Action 方法。
    5.  **异步执行同步 Action:** 使用 `await asyncio.to_thread(tool_action_method, arguments=arguments)` 来调用实际的 Action 方法。这允许同步的 Action 代码在不阻塞主事件循环的情况下运行。
    6.  **结果验证与处理:**
        *   检查 Action 返回的是否是包含 `status` 和 `message` 的标准字典。如果结构无效，强制标记为失败。
        *   **核心逻辑 - 提前退出:** 检查返回字典中的 `status` 字段。**如果 `status != "success"`，则记录失败结果，并立即 `break` 循环，不再执行计划中剩余的任何工具。**
    7.  **记录执行结果:** 将每个*尝试执行*的工具的结果（包含 `tool_call_id` 和 Action 返回的字典）收集到一个列表中。
    8.  **返回结果列表:** 将包含实际执行结果（可能因提前退出而不完整）的列表返回给 `CircuitDesignAgentV7`。
    9.  **错误处理:** 捕获参数 JSON 解析错误、Action 方法未找到 (`AttributeError`)、调用 Action 时参数不匹配 (`TypeError`) 或 Action 内部抛出的其他异常。将这些错误包装成标准的失败结果字典。
*   **关键方法:**
    *   `__init__(...)`: 接收 `CircuitDesignAgentV7` 实例，以便能访问其 Action 方法。
    *   `async execute_tool_calls(...)`: 实现上述核心职责的异步方法，包含迭代、调用、状态检查、提前退出和错误处理逻辑。

---

### 3.6. Action 方法 (位于 `CircuitDesignAgentV7` 内部 - 具体工具实现)

*   **角色:** 代表 Agent 可以执行的具体原子操作，是 Agent 与“世界”（在这里是电路状态 `MemoryManager.circuit_knowledge`）交互的接口。
*   **核心要求:**
    1.  **同步函数:** 在 V7 中，它们是普通的 Python 同步函数 `def method_name(...)`。
    2.  **标准输入:** 接收一个名为 `arguments` 的字典，包含从 LLM 规划中解析出的该工具所需参数。
    3.  **交互与状态修改:** 实现具体操作逻辑，通常需要读取或修改 `self.memory_manager.circuit_knowledge` 中的数据（例如，添加元件、检查元件是否存在、添加连接）。
    4.  **输入验证:** 在方法内部进行必要的参数验证（例如，检查 ID 是否存在、格式是否有效）。
    5.  **标准化输出:** **必须** 返回一个字典，至少包含：
        *   `status`: 字符串，值为 `"success"` 或 `"failure"`。
        *   `message`: 字符串，描述操作结果的自然语言消息（给 LLM 和开发者看）。
        *   可选 `data`: 字典，包含操作成功时产生的关键数据（如新元件的 ID）。
        *   可选 `error`: 字典，包含操作失败时的错误类型和详细信息。
*   **示例 (`connect_components_tool`):**
    *   接收 `arguments={'comp1_id': 'R1', 'comp2_id': 'B1'}`。
    *   验证 `comp1_id` 和 `comp2_id` 非空。
    *   检查 `self.memory_manager.circuit_knowledge['components']` 中是否存在 'R1' 和 'B1'。如果任一不存在，返回 `{"status": "failure", "message": "错误: 元件 'X' 不存在...", "error": ...}`。
    *   如果都存在，将排序后的元组 `('B1', 'R1')` 添加到 `self.memory_manager.circuit_knowledge['connections']` 集合中。
    *   返回 `{"status": "success", "message": "操作成功: 已将元件 'R1' 与 'B1' 连接起来...", "data": ...}`。

## 4. 交互流程 (`process_user_request` 详细步骤)

以下是处理单次用户请求的更详细的端到端流程：

1.  **[Agent] 输入接收与验证:**
    *   接收 `user_request` 字符串。
    *   检查是否为空或仅空白，若是则直接返回提示信息。

2.  **[Agent: Memory] 感知 - 更新短期记忆:**
    *   构造用户消息字典 `{"role": "user", "content": user_request}`。
    *   调用 `self.memory_manager.add_to_short_term()` 将其添加到列表末尾。
    *   (MemoryManager 内部) 如果列表长度超限，移除最旧的非系统消息。
    *   **异常处理:** 如果添加记忆失败，记录错误并返回错误信息给用户。

3.  **[Agent: Prep] 规划准备:**
    *   调用 `self.memory_manager.get_memory_context_for_prompt()` 获取当前电路状态和近期长期记忆的字符串。
    *   调用 `self._get_tool_schemas_for_prompt()` 获取可用工具描述字符串。
    *   调用 `self._get_planning_prompt_v7()` 构建包含上述上下文和严格 JSON 输出指令的 System Prompt。
    *   构建 `messages_for_llm1` 列表：包含 `[{"role": "system", "content": planning_prompt}, ...]` 加上 `memory_manager.short_term` 中的所有非系统消息。

4.  **[Agent -> LLM] 阶段 1: 规划 (含重试循环):**
    *   初始化 `attempt = 0`, `max_attempts = 1 + self.planning_llm_retries`。
    *   **进入循环 (while attempt < max_attempts):**
        *   `attempt += 1`
        *   **(Async Call)** 调用 `await self.llm_interface.call_llm(messages=messages_for_llm1)`。
            *   (LLMInterface 内部) `asyncio.to_thread` 执行同步 SDK 调用。
            *   捕获 API 级别的异常 (如 `ConnectionError`)。如果发生异常，记录错误，设置 `parser_error_msg`，如果已达最大尝试次数则跳出循环，否则继续下一次尝试。
        *   获取 LLM 响应对象 `first_llm_response`。检查其有效性（非 None, 有 choices）。若无效，设置错误信息并跳出循环。
        *   提取响应消息 `response_message = first_llm_response.choices[0].message`。
        *   **(Parse & Validate)** 调用 `thinking_process, plan_dict, parser_error_msg = self.output_parser.parse_planning_response(response_message)`。
        *   **检查结果:**
            *   如果 `plan_dict` 非 `None` 且 `parser_error_msg` 为空，说明解析和验证**成功** -> `break` 跳出重试循环。
            *   如果解析或验证失败，记录警告信息。如果 `attempt < max_attempts`，循环继续，进行下一次尝试。
    *   **循环结束:**
        *   如果 `plan_dict` 仍然是 `None` (所有尝试均失败)，记录严重错误，格式化包含最后错误信息的回复，并 `return` 给用户。

5.  **[Agent: Memory] 记录规划结果:**
    *   如果规划成功 (`plan_dict` 非 `None`)。
    *   获取第一次 LLM 响应的原始内容 `assistant_raw_content = first_llm_response.choices[0].message.content` (包含 `<think>` 和 JSON)。
    *   构造助手消息 `{"role": "assistant", "content": assistant_raw_content}`。
    *   调用 `self.memory_manager.add_to_short_term()` 存储。

6.  **[Agent] 决策: 工具调用 vs. 直接回复:**
    *   检查 `plan_dict.get("is_tool_calls")` 的值。

7.  **[Agent] 分支 A: 执行工具 (`is_tool_calls: true`)**
    *   **7a. [Prep] 准备工具执行:**
        *   从 `plan_dict['tool_list']` 获取工具列表。验证它是一个非空列表。
        *   按 `index` 字段对列表进行排序。检查索引是否连续（可选警告）。
        *   遍历排序后的列表，将每个工具项转换为 `ToolExecutor` 需要的模拟 ToolCall 字典格式（包含 `id`, `function.name`, `function.arguments` (JSON 字符串)）。
    *   **7b. [Executor Call] 执行 Action (Async Call):**
        *   调用 `tool_execution_results = await self.tool_executor.execute_tool_calls(mock_tool_calls_for_executor)`。
        *   (ToolExecutor 内部) 按顺序执行每个工具，检查 `status`，可能因失败而**提前退出**。返回实际执行的结果列表 `tool_execution_results`。
    *   **7c. [Memory] 观察 - 更新记忆:**
        *   遍历 `tool_execution_results`。
        *   对每个结果，构造 `tool` 角色消息 `{"role": "tool", "tool_call_id": result['tool_call_id'], "content": json.dumps(result['result'])}`。
        *   调用 `self.memory_manager.add_to_short_term()` 存储每个工具结果。
    *   **7d. [Prep] 报告准备:**
        *   调用 `self.memory_manager.get_memory_context_for_prompt()` 获取更新后的上下文。
        *   判断 `tools_were_skipped = len(tool_execution_results) < len(mock_tool_calls_for_executor)`。
        *   调用 `self._get_response_generation_prompt_v7()` 构建报告阶段的 System Prompt (传入 `tools_were_skipped` 信息)。
        *   构建 `messages_for_llm2` 列表：包含报告系统提示 + `memory_manager.short_term` 中的所有消息（现在包括 user, assistant-plan, tool-results）。
    *   **7e. [LLM Call] 阶段 2: 回复生成 (Async Call):**
        *   调用 `second_llm_response = await self.llm_interface.call_llm(messages=messages_for_llm2)`。
        *   捕获可能的 API 异常。
    *   **7f. [Parse] 解析最终报告:**
        *   检查 `second_llm_response` 是否有效。若无效，生成错误报告。
        *   若有效，提取 `raw_final_content = second_llm_response.choices[0].message.content`。
        *   调用 `final_thinking, final_reply = self.output_parser._parse_llm_text_content(raw_final_content)`。
    *   **7g. [Memory] 记录最终报告:**
        *   构造最终助手消息（可以使用原始 `second_llm_response.choices[0].message.model_dump()` 或解析后的 `think`+`reply` 格式）。
        *   调用 `self.memory_manager.add_to_short_term()` 存储。
    *   **7h. [Return] 返回结果:**
        *   格式化最终报告 `f"<think>{final_thinking}</think>\n\n{final_reply}"`。
        *   `return` 该字符串。

8.  **[Agent] 分支 B: 直接回复 (`is_tool_calls: false`)**
    *   **8a. [Extract] 获取直接回复:**
        *   从 `plan_dict` 中获取 `direct_reply = plan_dict.get("direct_reply")`。
        *   验证 `direct_reply` 是否为有效的非空字符串。如果无效，生成错误信息替代。
    *   **8b. [Format] 格式化输出:**
        *   使用第一次 LLM 调用产生的 `thinking_process`。
        *   组合 `f"<think>{thinking_process}</think>\n\n{direct_reply}"`。
    *   **8c. [Return] 返回结果:**
        *   `return` 该字符串。

## 5. 示例交互流程 (更详尽的注释)

---

### 流程 1: LLM API 调用失败

1.  **用户:** "给我加个灯 L1"
2.  **Agent:** 记录用户消息到短期记忆。准备好包含系统提示和用户消息的 `messages_for_llm1`。
3.  **Agent -> LLM (规划 - 尝试 1):** 调用 `await self.llm_interface.call_llm(messages_for_llm1)`。
4.  **LLM API / SDK:** 失败！可能是网络超时、API Key 错误、服务宕机等。`zhipuai` SDK 抛出异常 (如 `ApiException`, `ConnectionError`)。
5.  **LLMInterface:** `call_llm` 方法中的 `try...except` 块捕获到这个异常。
6.  **Agent:** `process_user_request` 中调用 `call_llm` 的地方收到这个异常。由于是 API 层面错误，重试可能也无效（或按策略只重试特定错误）。假设直接进入失败处理。
7.  **Agent:** 检测到 `plan_dict` 为 `None`，且有 `parser_error_msg` (内容是 API 错误信息)。
8.  **Agent -> 用户:** 返回格式化的错误报告，清晰说明是与 LLM 通信时出错。

---

### 流程 2: 规划 JSON 解析失败 -> 重试 -> 成功

1.  **用户:** "添加电容 C1 和 电阻 R1"
2.  **Agent:** 记录用户消息。准备规划提示。
3.  **Agent -> LLM (规划 - 尝试 1):** `await call_llm(...)`。
4.  **LLM -> Agent:** 返回 `<think>好的，计划添加 C1 和 R1...</think> { "is_tool_calls": true, "tool_list": [ { "toolname": "add_component_tool", "params": {"component_type": "capacitor", "component_id": "C1"}, "index": 1 }, { "toolname": "add_component_tool", "params": {"component_type": "resistor", "component_id": "R1"}, "index": 2 ] "direct_reply": null }` **<-- 注意：tool_list 的末尾缺少逗号，JSON 无效！**
5.  **Agent: Parser:** `parse_planning_response` 尝试 `json.loads()`，失败，抛出 `JSONDecodeError`。解析器捕获此错误，设置 `parser_error_msg = "解析 JSON 失败: ..."`, 返回 `plan_dict = None`。
6.  **Agent:** 在重试循环中，检查到 `plan_dict` 为 `None` 且 `attempt < max_attempts`。准备进行第二次尝试。
7.  **Agent -> LLM (规划 - 尝试 2):** 再次调用 `await call_llm(...)`。
8.  **LLM -> Agent:** 这次返回了**格式正确**的 JSON。
    ```json
    {
        "is_tool_calls": true,
        "tool_list": [
            { "toolname": "add_component_tool", "params": {"component_type": "capacitor", "component_id": "C1"}, "index": 1 },
            { "toolname": "add_component_tool", "params": {"component_type": "resistor", "component_id": "R1"}, "index": 2 }
        ],
        "direct_reply": null
    }
    ```
9.  **Agent: Parser:** `parse_planning_response` 成功解析并验证通过，返回有效的 `plan_dict`。
10. **Agent:** `break` 退出重试循环。记录助手的原始规划响应到内存。
11. **Agent:** 检测到 `is_tool_calls: true`，进入**分支 A (执行工具)**。后续流程类似【流程 4: 多个工具成功】。

---

### 流程 3: 单个工具成功 (以 `describe_circuit_tool` 为例)

1.  **用户:** "电路现在什么样子？"
2.  **Agent -> LLM (规划):** LLM 分析后认为需要调用 `describe_circuit_tool`。返回正确的 JSON 计划：
    ```json
    {
        "is_tool_calls": true,
        "tool_list": [
            { "toolname": "describe_circuit_tool", "params": {}, "index": 1 }
        ],
        "direct_reply": null
    }
    ```
3.  **Agent:** 解析成功。记录规划。进入分支 A。
4.  **Agent: Prep:** 准备模拟调用 `[{ "id": "plan1-...", "function": {"name": "describe_circuit_tool", "arguments": '{}'} }]`。
5.  **Agent -> Executor:** `await tool_executor.execute_tool_calls(...)`。
6.  **Executor:**
    *   找到 `agent.describe_circuit_tool` 方法。
    *   调用 `await asyncio.to_thread(agent.describe_circuit_tool, arguments={})`。
    *   **Action `describe_circuit_tool`:** 调用 `self.memory_manager.get_circuit_state_description()` 获取描述文本。返回 `{"status": "success", "message": "已成功获取...", "data": {"description": "老板，这是当前电路的状态报告：\n  - 元件 (0): (无)\n  - 连接 (0): (无)"}}` (假设电路为空)。
    *   Executor 收到 `status: "success"`。记录结果。
7.  **Executor -> Agent:** 返回包含成功结果的列表。
8.  **Agent: Memory:** 将 `describe_circuit_tool` 的成功结果（包含描述数据）作为 `role: tool` 消息存入内存。
9.  **Agent: Prep:** 准备报告提示。
10. **Agent -> LLM (报告):** 发送包含用户请求、规划、工具结果的完整历史。
11. **LLM -> Agent:** 生成报告，引用工具结果中的描述: `<think>用户问电路状态。规划调用 describe_circuit_tool。工具成功执行并返回了描述。现在需要将这个描述呈现给用户。</think>\n\n老板，这是当前电路的状态报告：\n  - 元件 (0): (无)\n  - 连接 (0): (无)`
12. **Agent:** 解析报告，存入内存，返回给用户。

---

### 流程 4: 多个工具成功 (详细步骤)

1.  **用户:** "加个开关 SW1，再加个 LED L1，然后把 SW1 和 L1 连起来"
2.  **Agent -> LLM (规划):** 成功获取包含三个步骤 (`add_component` SW1, `add_component` L1, `connect_components` SW1-L1) 的有效 JSON 计划。
3.  **Agent:** 解析成功，记录规划，进入分支 A。
4.  **Agent: Prep:** 准备三个模拟工具调用。
5.  **Agent -> Executor:** `await tool_executor.execute_tool_calls(...)`
6.  **Executor (迭代 1 - add SW1):**
    *   调用 `agent.add_component_tool` (for SW1)。
    *   Action 成功，返回 `{"status": "success", ...}`。
    *   Executor 记录结果 1。
7.  **Executor (迭代 2 - add L1):**
    *   调用 `agent.add_component_tool` (for L1)。
    *   Action 成功，返回 `{"status": "success", ...}`。
    *   Executor 记录结果 2。
8.  **Executor (迭代 3 - connect SW1-L1):**
    *   调用 `agent.connect_components_tool` (for SW1, L1)。
    *   Action 检查 SW1 和 L1 都存在，执行连接。返回 `{"status": "success", ...}`。
    *   Executor 记录结果 3。
9.  **Executor:** 完成所有计划工具，无失败。
10. **Executor -> Agent:** 返回包含三个成功结果的列表。
11. **Agent: Memory:** 将三个 `role: tool` 成功消息添加到短期记忆。
12. **Agent -> LLM (报告):** 发送完整历史。
13. **LLM -> Agent:** 生成总结报告，确认所有步骤成功完成。
14. **Agent:** 解析、存储、返回最终报告给用户。

---

### 流程 5: 多个工具，一个失败导致提前退出 (详细步骤)

1.  **用户:** "添加电池 B2。连接 B2 和 R99 (R99不存在)。添加地线 G1。"
2.  **Agent -> LLM (规划):** 成功获取三步计划 (add B2, connect B2-R99, add G1) 的 JSON。
3.  **Agent:** 解析成功，记录规划，进入分支 A。
4.  **Agent: Prep:** 准备三个模拟工具调用。
5.  **Agent -> Executor:** `await tool_executor.execute_tool_calls(...)`。
6.  **Executor (迭代 1 - add B2):**
    *   调用 `agent.add_component_tool` (for B2)。
    *   Action 成功，返回 `{"status": "success", ...}`。
    *   Executor 记录结果 1。
7.  **Executor (迭代 2 - connect B2-R99):**
    *   调用 `agent.connect_components_tool` (for B2, R99)。
    *   **Action `connect_components_tool`:** 内部检查发现 `R99` 不在 `self.memory_manager.circuit_knowledge['components']` 中。
    *   Action 返回 `{"status": "failure", "message": "错误: 无法连接：元件 'R99' 不存在。", "error": ...}`。
    *   **Executor:** 在 `for mock_call in mock_tool_calls:` 循环内部，检查到返回的 `action_result.get("status") != "success"`。
    *   **关键:** 执行 `logger.warning(...)` 记录失败，并执行 `break` 语句！
8.  **Executor:** **循环提前终止！** 不会再处理计划中的第三个工具 (add G1)。
9.  **Executor -> Agent:** 返回一个**只包含两个结果**的列表 `[result_B2_success, result_connect_failure]`。
10. **Agent: Memory:** 将这两个结果（一个成功，一个失败）作为 `role: tool` 消息存入短期记忆。
11. **Agent: Prep:**
    *   获取上下文。
    *   计算 `tools_were_skipped = True` (因为 2 < 3)。
    *   构建报告系统提示，传入 `tools_were_skipped=True`，指示 LLM 需要解释跳过的步骤。
12. **Agent -> LLM (报告):** 发送历史记录，其中包含 B2 添加成功的 `tool` 消息和连接 B2-R99 失败的 `tool` 消息。
13. **LLM -> Agent:** 生成报告，必须解释为何 G1 未添加: `<think>用户要求添加 B2，连接 B2-R99，添加 G1。计划了三步。第一步添加 B2 成功。第二步连接 B2-R99 失败，因为 R99 不存在。由于第二步失败，执行器提前中止，第三步添加 G1 未执行。必须在报告中清晰说明这一点。</think>\n\n老板，我已经成功添加了电池 B2。但是在尝试连接 B2 与 R99 时，操作失败了，原因是元件 'R99' 目前不存在。因此，计划中后续添加地线 G1 的步骤没有被执行。如果您需要连接 R99，请先添加它。`
14. **Agent:** 解析、存储、返回这份包含失败原因和未执行步骤说明的报告。

---

### 流程 6: 直接回复 (无需工具)

1.  **用户:** "你好" 或 "谢谢你"
2.  **Agent -> LLM (规划):** LLM 分析认为这是简单问候或致谢，无需操作电路。返回 JSON 计划：
    ```json
    {
        "is_tool_calls": false,
        "tool_list": null,
        "direct_reply": "您好，老板！有什么可以帮您的吗？" // 或 "不客气，老板！"
    }
    ```
3.  **Agent:** 解析成功。记录规划。
4.  **Agent:** 检测到 `is_tool_calls: false`。进入**分支 B (直接回复)**。
5.  **Agent: Extract:** 从计划中获取 `direct_reply` 字符串。
6.  **Agent: Format:** 使用第一次 LLM 的思考过程 (`thinking_process` 来自 `parse_planning_response`) 和 `direct_reply` 组合最终输出。
7.  **Agent -> 用户:** 返回 `"<think>用户只是打个招呼/表示感谢，无需工具调用，直接回复即可。</think>\n\n您好，老板！有什么可以帮您的吗？"`。

## 6. Prompts 的核心思想

*   **规划 Prompt (`_get_planning_prompt_v7`):**
    *   **核心指令:** 输出 `<think>` + **严格格式的单一 JSON 对象**。
    *   **禁止:** 明确禁止使用 `tool_calls`。
    *   **JSON 结构:** 详细定义 `is_tool_calls`, `tool_list` (及其内部结构 `toolname`, `params`, `index`), `direct_reply` 的类型和依赖关系。
    *   **上下文:** 注入可用工具的 schema 和当前的记忆上下文（电路状态+近期经验）。
    *   **强调:** 反复强调格式的绝对重要性。
*   **报告 Prompt (`_get_response_generation_prompt_v7`):**
    *   **核心任务:** 基于**完整的历史记录**（特别是 `role: tool` 消息）生成总结性回复。
    *   **关键要求:**
        *   **必须**关注 `tool` 消息中的 `status` 和 `message` (以及 `error`)。
        *   清晰总结**所有已执行**工具的成功与失败。
        *   如果 `tools_were_skipped` (由 Agent 判断并传入提示)，**必须**在报告中解释为何后续步骤未执行。
    *   **输出格式:** 要求 `<think>` + `\n\n` + 正式回复文本。
    *   **禁止:** 不要再次生成 `tool_calls`。
    *   **上下文:** 提供更新后的记忆上下文和工具列表作为参考。

## 7. 未来方向 / 局限性 (补充说明)

*   **Token-Based Pruning:** 需要引入一个分词器（tokenizer，例如 Tiktoken）来计算消息的 token 数量，然后基于总 token 数而不是消息条数进行修剪，这样更精确。
*   **RAG for Long-Term Memory:** 实现方式大致为：
    1.  将长期记忆的知识片段（以及可能的对话摘要）用 Embedding 模型转换为向量，存入向量数据库。
    2.  当需要上下文时，将当前用户查询（或相关对话片段）也转换为向量。
    3.  在向量数据库中执行相似性搜索，找出与当前查询最相关的 N 条长期记忆。
    4.  将这些检索到的相关记忆片段注入到 LLM 的 Prompt 中，而不是仅仅使用最近的 N 条。
*   **高级错误恢复:** 例如，如果 `connect_components_tool` 失败因为元件不存在，Agent 可以不仅仅是报告失败，而是*在同一轮内*自动规划一个新的 `add_component_tool` 步骤来添加缺失的元件，然后再尝试连接（这会显著增加 Agent 逻辑的复杂度）。
*   **异步 Actions:** 如果 `add_component_tool` 需要调用一个外部（慢速）的零件数据库 API 来验证元件类型，那么 `add_component_tool` 本身就应该被写成 `async def`，内部使用 `await http_client.get(...)` 等异步 I/O 操作。然后在 `ToolExecutor` 中直接 `await self.agent_instance.add_component_tool(...)` 调用它。

## 8. 如何运行 (再次确认)

1.  **设置 API Key:** 确保 `ZHIPUAI_API_KEY` 环境变量已设置，或者准备好在脚本提示时手动输入。
2.  **安装依赖:** 确保已安装 `zhipuai` (以及 Python 基础环境)。 `pip install zhipuai`
3.  **执行脚本:** 在终端中运行 `python openmanus_v7_base.py`。
4.  **交互:** 等待 "Boss V7.0，请指示！" 提示出现后，输入您的指令。使用 '退出' 或 'quit' 等命令结束。

老板，这份超级详细的版本应该把 V7 的里里外外都讲透彻了。如果您还有任何疑问，随时提出！